# Network analysis in Senegal

### Objectives
    1)	Use measures of road-based accessibility to identify road segments that, if rehabilitated, would improve agricultural market activities in Senegal, including during flood conditions.
    2)	Gain a better understanding of the accessibility, connectivity, and criticality of roads in Senegal in relationship to agricultural origins, processing & transfer sites, and markets.

To this end, the team will develop an accessibility model which measures the travel time from sites of agricultural production to their nearest populated areas, processing centers, and markets. 

### Datasets for analysis
#### ORIGIN
    1) agriculture: MapSPAM 2017. Measuring value in international dollars.
    2) agriculture: UMD Land Cover 2019 30m. Assign MapSPAM value onto land cover cropland class for more precise origin information.
    3) population: WorldPop 2020, UN-adjusted.
    4) settlement extent: GRID3 2020.
#### DESTINATION
    4) markets: derived from WorldPop 2020 and GRID3 2020 urban clusters.
    5) agricultural processing hubs: to be acquired.
#### TRAVEL ROUTE
    6) roads: OpenStreetMap, July 2021.
    7) elevation: 
#### OBSTACLE
    8) flood: FATHOM. 1-in-10, 20, and 50 year flood return periods. 
#### INTERVENTION
    9) upcoming road projects: AGEROUTE interventions separate from the World Bank-financed project
    10) targeted road projects: critical road segments identified by this accessibility model's baseline outputs


### Model design
#### Basic formula: 
    (a) Off-road driving time from origin to closest road node
    +
    (b) Driving time from road node in (a) to a destination (closeness measured by road segments speeds)

#### Model origin & destination (OD) sets:
    A)	Travel time from an area that has agricultural value/potential to the nearest processing hub (if provided).
    B)	Travel time from an area that has agricultural value/potential to the nearest larger settlement, (“larger” settlement identified using a case-appropriate population metric to be determined).
    C)	Travel time from an area that has agricultural value/potential to the nearest market.
    D)	Travel time from all settlements to the nearest market.
    E)	Travel time from larger settlements to the nearest market.

#### Before/after scenarios for each OD set:
    1)	Pre-project, baseline weather: No inclement weather. Road network status as of November 2021.
    2)	Pre-project, flood: 1-in-10, 1-in-20 and 1-in-50 year flood return period. Road network status as of November 2021.
    3)	Post-project, baseline weather: No inclement weather. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).
    4)	Post-project, flood: 1-in-10 year flood return period. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).

#### Notes:
    --Destinations are expected to be proximal to the road network, so no measure is taken between road and destination.
    --All travel times will be assigned to each model variation’s point of origin; the aggregation up to admin areas is possible if desired.
    --Obstacles & interventions modify the road segment speeds. Basic formula is then applied to the modified road network.


### Prep workspace

In [1]:
import os, sys
GISFolder = os.getcwd()
GISFolder

'C:\\Users\\wb527163\\GEO-Cdrive-Grace'

In [2]:
# Note: needed to reinstall rtree due to geopandas import error. Did so in the console. 
# conda install -c conda-forge rtree=0.9.3

In [3]:
# load and filter osm network (step 1)
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import time
sys.path.append(r"C:\Users\wb527163\.conda\envs\geo\GOSTnets-master")
import GOSTnets as gn

In [4]:
import networkx as nx
import osmnx as ox
import numpy as np
import rasterio as rt
import shapely
from shapely.geometry import Point, box
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely import wkt
from shapely.geometry import LineString, MultiLineString, Point
import peartree

In [5]:
#### Might not use these
import fiona
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [6]:
pth = os.path.join(GISFolder, "SEN-Cdrive") # Personal folder system for running model.
pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive'

In [7]:
out_pth = os.path.join(GISFolder, "SEN-Cdrive\outputs") # For storing intermediate outputs from the model.
out_pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive\\outputs'

In [8]:
team_pth = 'R:\\SEN\\GEO' # This is where the unmodified input data is stored. Finalized outputs also housed here.
team_pth

'R:\\SEN\\GEO'

### Prepare and clean the data

#### Return periods from FATHOM: 1-in-10, 20, and 50 year floods.

We are starting with just the 1-in-10 year return period. Joining 1-in-20 and 1-in-50 to the road dataframe was causing file size errors. 20 and 50 will be run in a replicated script.

In [42]:
flood50 = gpd.read_file("C:/Users/wb527163/GEO-Cdrive-Grace/SEN-Cdrive/scratch.gdb", layer="PFU_1in50")
flood50.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2459356 entries, 0 to 2459355
Data columns (total 4 columns):
 #   Column        Dtype   
---  ------        -----   
 0   PFU_1in50     float64 
 1   Shape_Length  float64 
 2   Shape_Area    float64 
 3   geometry      geometry
dtypes: float64(3), geometry(1)
memory usage: 75.1 MB


In [20]:
gTime = nx.read_gpickle("SEN-Cdrive/gTime.pickle")
gTime_edge = gn.edge_gdf_from_graph(gTime)
gTime_edge

,stnode,endnode,osmid,mode,highway,time,ref,length,access,oneway,bridge,lanes,junction,tunnel,width,maxspeed,name,area,geometry
0,358284990,5217543379,59618174,drive,unclassified,2.385144,D 523,33.127,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,D 523,NaN,"LINESTRING (-12.32347 12.38119, -12.32368 12.3..."
1,358284990,1888282175,178482063,drive,tertiary,0.769920,NaN,12.832,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-12.32347 12.38119, -12.32351 12.3..."
2,358284990,5329792467,178482063,drive,tertiary,2.926860,NaN,48.781,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-12.32347 12.38119, -12.32317 12.3..."
3,358284993,1888282575,178470940,drive,tertiary,7.594620,NaN,126.577,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-12.28135 12.41380, -12.28205 12.4..."
4,358284993,1888198886,178470940,drive,tertiary,7.234680,NaN,120.578,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-12.28135 12.41380, -12.28061 12.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4012560,9246539941,9246539942,366052716,drive,residential,6.906780,NaN,76.742,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-17.48521 14.72445, -17.48451 14.7..."
4012561,9246539942,9246539941,366052716,drive,residential,6.906780,NaN,76.742,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-17.48451 14.72456, -17.48521 14.7..."
4012562,9246539942,3700438702,366052716,drive,residential,0.076140,NaN,0.846,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-17.48451 14.72456, -17.48451 14.7..."
4012563,9276108905,6048975958,177950649,drive,secondary,8.840674,NaN,171.902,NaN,True,NaN,2,NaN,NaN,NaN,NaN,Route de l'Aeroport,NaN,"LINESTRING (-17.50499 14.74971, -17.50357 14.7..."


In [11]:
gTime_node = os.path.join(pth, "gTime_nodes.csv")
gTime_node = pd.read_csv(gTime_node)
gTime_node.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1829568 entries, 0 to 1829567
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   node_ID     int64  
 2   y           float64
 3   highway     object 
 4   x           float64
 5   ref         object 
 6   geometry    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 97.7+ MB


C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Update driving times based on flood intersection.

#### Join road network and flood raster into single table.

In [26]:
gTime_edge.reset_index(inplace=True) # To create unique ID and to avoid:  ValueError: cannot reindex from a duplicate axis.
gTime_edge.rename(columns={'index': 'ID_graph'}, inplace=True)
gTime_edge.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4012565 entries, 0 to 4012564
Data columns (total 20 columns):
 #   Column    Dtype   
---  ------    -----   
 0   ID_graph  int64   
 1   stnode    int64   
 2   endnode   int64   
 3   osmid     int64   
 4   mode      object  
 5   highway   object  
 6   time      float64 
 7   ref       object  
 8   length    float64 
 9   access    object  
 10  oneway    bool    
 11  bridge    object  
 12  lanes     object  
 13  junction  object  
 14  tunnel    object  
 15  width     object  
 16  maxspeed  object  
 17  name      object  
 18  area      object  
 19  geometry  geometry
dtypes: bool(1), float64(2), geometry(1), int64(4), object(12)
memory usage: 585.5+ MB


In [43]:
# Spatial join should be on projected GDFs.
gTime_edge = gTime_edge.to_crs("EPSG:31028")
flood50 = flood50.to_crs("EPSG:31028")
gTime_edge.crs == flood50.crs

True

In [44]:
join50 = gpd.sjoin_nearest(gTime_edge, flood50, how="left", max_distance=3) 
join50.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4195759 entries, 0 to 4012564
Data columns (total 24 columns):
 #   Column        Dtype   
---  ------        -----   
 0   ID_graph      int64   
 1   stnode        int64   
 2   endnode       int64   
 3   osmid         int64   
 4   mode          object  
 5   highway       object  
 6   time          float64 
 7   ref           object  
 8   length        float64 
 9   access        object  
 10  oneway        bool    
 11  bridge        object  
 12  lanes         object  
 13  junction      object  
 14  tunnel        object  
 15  width         object  
 16  maxspeed      object  
 17  name          object  
 18  area          object  
 19  geometry      geometry
 20  index_right   float64 
 21  PFU_1in50     float64 
 22  Shape_Length  float64 
 23  Shape_Area    float64 
dtypes: bool(1), float64(6), geometry(1), int64(4), object(12)
memory usage: 772.3+ MB


In [45]:
join50

,ID_graph,stnode,endnode,osmid,mode,highway,time,ref,length,access,...,tunnel,width,maxspeed,name,area,geometry,index_right,PFU_1in50,Shape_Length,Shape_Area
0,0,358284990,5217543379,59618174,drive,unclassified,2.385144,D 523,33.127,NaN,...,NaN,NaN,NaN,D 523,NaN,"LINESTRING (790871.607 1370083.257, 790848.290...",NaN,NaN,NaN,NaN
1,1,358284990,1888282175,178482063,drive,tertiary,0.769920,NaN,12.832,NaN,...,NaN,NaN,NaN,NaN,NaN,"LINESTRING (790871.607 1370083.257, 790867.287...",NaN,NaN,NaN,NaN
2,2,358284990,5329792467,178482063,drive,tertiary,2.926860,NaN,48.781,NaN,...,NaN,NaN,NaN,NaN,NaN,"LINESTRING (790871.607 1370083.257, 790904.086...",NaN,NaN,NaN,NaN
3,3,358284993,1888282575,178470940,drive,tertiary,7.594620,NaN,126.577,NaN,...,NaN,NaN,NaN,NaN,NaN,"LINESTRING (795418.079 1373739.218, 795343.307...",NaN,NaN,NaN,NaN
4,4,358284993,1888198886,178470940,drive,tertiary,7.234680,NaN,120.578,NaN,...,NaN,NaN,NaN,NaN,NaN,"LINESTRING (795418.079 1373739.218, 795497.547...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4012560,4012560,9246539941,9246539942,366052716,drive,residential,6.906780,NaN,76.742,NaN,...,NaN,NaN,NaN,NaN,NaN,"LINESTRING (232231.532 1629246.092, 232307.524...",NaN,NaN,NaN,NaN
4012561,4012561,9246539942,9246539941,366052716,drive,residential,6.906780,NaN,76.742,NaN,...,NaN,NaN,NaN,NaN,NaN,"LINESTRING (232307.524 1629257.675, 232231.532...",NaN,NaN,NaN,NaN
4012562,4012562,9246539942,3700438702,366052716,drive,residential,0.076140,NaN,0.846,NaN,...,NaN,NaN,NaN,NaN,NaN,"LINESTRING (232307.524 1629257.675, 232307.393...",NaN,NaN,NaN,NaN
4012563,4012563,9276108905,6048975958,177950649,drive,secondary,8.840674,NaN,171.902,NaN,...,NaN,NaN,NaN,Route de l'Aeroport,NaN,"LINESTRING (230131.706 1632066.089, 230285.874...",NaN,NaN,NaN,NaN


In [46]:
# How many nodes experienced flooding?
pc_flooded = join50["PFU_1in50"].count() / len(join50) * 100

print("No flood crossing at node:", join50["PFU_1in50"].isnull().sum(), "locations", end="\n")
print("Flood crossing at node:", join50["PFU_1in50"].count(), "locations", end="\n")
print("\nPercent flooded:", pc_flooded, "percent", "out of", len(join50), "possible locations")

No flood crossing at node: 3748839 locations
Flood crossing at node: 446920 locations

Percent flooded: 10.651708069982094 percent out of 4195759 possible locations


In [47]:
join50 = join50[['ID_graph', 'stnode', 'endnode', 'time', 'length', 'highway', 'osmid', 'geometry', 'PFU_1in50']]
join50.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4195759 entries, 0 to 4012564
Data columns (total 9 columns):
 #   Column     Dtype   
---  ------     -----   
 0   ID_graph   int64   
 1   stnode     int64   
 2   endnode    int64   
 3   time       float64 
 4   length     float64 
 5   highway    object  
 6   osmid      int64   
 7   geometry   geometry
 8   PFU_1in50  float64 
dtypes: float64(3), geometry(1), int64(4), object(1)
memory usage: 320.1+ MB


In [48]:
join50 = join50.to_crs("EPSG:4326")
join50.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [49]:
join50

,ID_graph,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in50
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,"LINESTRING (-12.32347 12.38119, -12.32368 12.3...",NaN
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32351 12.3...",NaN
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32317 12.3...",NaN
3,3,358284993,1888282575,7.594620,126.577,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28205 12.4...",NaN
4,4,358284993,1888198886,7.234680,120.578,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28061 12.4...",NaN
...,...,...,...,...,...,...,...,...,...
4012560,4012560,9246539941,9246539942,6.906780,76.742,residential,366052716,"LINESTRING (-17.48521 14.72445, -17.48451 14.7...",NaN
4012561,4012561,9246539942,9246539941,6.906780,76.742,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48521 14.7...",NaN
4012562,4012562,9246539942,3700438702,0.076140,0.846,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48451 14.7...",NaN
4012563,4012563,9276108905,6048975958,8.840674,171.902,secondary,177950649,"LINESTRING (-17.50499 14.74971, -17.50357 14.7...",NaN


In [50]:
# Fewer errors farther down when using dataframe instead of gdf
join50_df = pd.DataFrame(join50)
join50_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4195759 entries, 0 to 4012564
Data columns (total 9 columns):
 #   Column     Dtype   
---  ------     -----   
 0   ID_graph   int64   
 1   stnode     int64   
 2   endnode    int64   
 3   time       float64 
 4   length     float64 
 5   highway    object  
 6   osmid      int64   
 7   geometry   geometry
 8   PFU_1in50  float64 
dtypes: float64(3), geometry(1), int64(4), object(1)
memory usage: 320.1+ MB


In [51]:
join50_df

,ID_graph,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in50
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,"LINESTRING (-12.32347 12.38119, -12.32368 12.3...",NaN
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32351 12.3...",NaN
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32317 12.3...",NaN
3,3,358284993,1888282575,7.594620,126.577,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28205 12.4...",NaN
4,4,358284993,1888198886,7.234680,120.578,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28061 12.4...",NaN
...,...,...,...,...,...,...,...,...,...
4012560,4012560,9246539941,9246539942,6.906780,76.742,residential,366052716,"LINESTRING (-17.48521 14.72445, -17.48451 14.7...",NaN
4012561,4012561,9246539942,9246539941,6.906780,76.742,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48521 14.7...",NaN
4012562,4012562,9246539942,3700438702,0.076140,0.846,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48451 14.7...",NaN
4012563,4012563,9276108905,6048975958,8.840674,171.902,secondary,177950649,"LINESTRING (-17.50499 14.74971, -17.50357 14.7...",NaN


In [52]:
join50_df.to_csv(os.path.join(out_pth, 'gTime_flood50_intermediate.csv'))

### Create speed penalties.
Note: Flood depths are in centimeters. FATHOM uses meters, but conversion process to vector required some finessing. 

In [53]:
# Give a depth to the nodes that don't cross a flood point. 
join50.loc[join50['PFU_1in50'].isnull(), 'PFU_1in50'] = -1

In [54]:
join50["t50"] = 1 # This is the penalty column.
join50.loc[join50['PFU_1in50'] < 0, 't50'] = 1 # Where no flood crosses, keep the default value (no penalty).
join50.loc[(join50['PFU_1in50'] > 10) & (join50['PFU_1in50'] <= 30), 't50'] = 1.25
join50.loc[(join50['PFU_1in50'] > 30) & (join50['PFU_1in50'] <= 60), 't50'] = 2
join50.loc[(join50['PFU_1in50'] > 60) & (join50['PFU_1in50'] <= 90), 't50'] = 5
join50.loc[(join50['PFU_1in50'] > 90), 't50'] = 9999
join50

,ID_graph,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in50,t50
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,"LINESTRING (-12.32347 12.38119, -12.32368 12.3...",-1.0,1.0
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32351 12.3...",-1.0,1.0
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32317 12.3...",-1.0,1.0
3,3,358284993,1888282575,7.594620,126.577,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28205 12.4...",-1.0,1.0
4,4,358284993,1888198886,7.234680,120.578,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28061 12.4...",-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
4012560,4012560,9246539941,9246539942,6.906780,76.742,residential,366052716,"LINESTRING (-17.48521 14.72445, -17.48451 14.7...",-1.0,1.0
4012561,4012561,9246539942,9246539941,6.906780,76.742,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48521 14.7...",-1.0,1.0
4012562,4012562,9246539942,3700438702,0.076140,0.846,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48451 14.7...",-1.0,1.0
4012563,4012563,9276108905,6048975958,8.840674,171.902,secondary,177950649,"LINESTRING (-17.50499 14.74971, -17.50357 14.7...",-1.0,1.0


In [55]:
# Turn the penalty column into a flood-affected time column.
join50['t50'] = join50['t50'] * join50['time']
join50

,ID_graph,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in50,t50
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,"LINESTRING (-12.32347 12.38119, -12.32368 12.3...",-1.0,2.385144
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32351 12.3...",-1.0,0.769920
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32317 12.3...",-1.0,2.926860
3,3,358284993,1888282575,7.594620,126.577,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28205 12.4...",-1.0,7.594620
4,4,358284993,1888198886,7.234680,120.578,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28061 12.4...",-1.0,7.234680
...,...,...,...,...,...,...,...,...,...,...
4012560,4012560,9246539941,9246539942,6.906780,76.742,residential,366052716,"LINESTRING (-17.48521 14.72445, -17.48451 14.7...",-1.0,6.906780
4012561,4012561,9246539942,9246539941,6.906780,76.742,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48521 14.7...",-1.0,6.906780
4012562,4012562,9246539942,3700438702,0.076140,0.846,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48451 14.7...",-1.0,0.076140
4012563,4012563,9276108905,6048975958,8.840674,171.902,secondary,177950649,"LINESTRING (-17.50499 14.74971, -17.50357 14.7...",-1.0,8.840674


In [56]:
join50.to_csv(os.path.join(out_pth, 'join50.csv'))

### Convert back to graph object.

In [9]:
# Converting back to graph can cause memory errors. Suggested to restart the kernel and reload the nodes and revised edges at this point.
gTime_node = os.path.join(pth, "gTime_nodes.csv")
gTime_node = pd.read_csv(gTime_node)
join50 = os.path.join(out_pth, "join50.csv")
join50 = pd.read_csv(join50)
print(gTime_node.info())
print(join50.info())

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1829568 entries, 0 to 1829567
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   node_ID     int64  
 2   y           float64
 3   highway     object 
 4   x           float64
 5   ref         object 
 6   geometry    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 97.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4195759 entries, 0 to 4195758
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   ID_graph    int64  
 2   stnode      int64  
 3   endnode     int64  
 4   time        float64
 5   length      float64
 6   highway     object 
 7   osmid       int64  
 8   geometry    object 
 9   PFU_1in50   float64
 10  t50         float64
dtypes: float64(4), int64(5), object(2)
memory usage: 352.1+ MB
None


In [10]:
print('start: %s\n' % time.ctime())
G_flood = gn.edges_and_nodes_gdf_to_graph(gTime_node, join50, node_tag='node_ID', u_tag='stnode', v_tag='endnode', geometry_tag='geometry')
gn.example_edge(G_flood, 10)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Wed Dec 22 13:12:40 2021

(358284990, 5217543379, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001411CF7FF40>, 'Unnamed: 0': 0, 'ID_graph': 0, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 'unclassified', 'osmid': 59618174, 'PFU_1in50': -1.0, 't50': 2.3851440000000004})
(358284990, 1888282175, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001411CF7FEB0>, 'Unnamed: 0': 1, 'ID_graph': 1, 'time': 0.76992, 'length': 12.832, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in50': -1.0, 't50': 0.76992})
(358284990, 5329792467, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001411CF7FE80>, 'Unnamed: 0': 2, 'ID_graph': 2, 'time': 2.92686, 'length': 48.781, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in50': -1.0, 't50': 2.92686})
(5217543379, 358284990, {'geometry': <shapely.geometry.linestring.LineString object at 0x0000014189C99340>, 'Unnamed: 0': 16698, 'ID_graph': 16698, 'time': 2.3851440000000004, 'len

In [11]:
print('start: %s\n' % time.ctime())
gn.save(G_flood, 'gTime_flood50', out_pth, edges = True, nodes = True)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Wed Dec 22 13:21:57 2021


end: Wed Dec 22 13:34:54 2021

--- processing complete


### Create travel time values for the road nodes nearest to each service.

Using calculate_OD.

In [9]:
# If starting a new session, load from file.
HDurban_snap = os.path.join(out_pth, "HDurban_snap.csv")
HDurban_snap = pd.read_csv(HDurban_snap)
hamlet_snap = os.path.join(out_pth, "hamlet_snap.csv")
hamlet_snap = pd.read_csv(hamlet_snap)

In [10]:
print('start: %s\n' % time.ctime())
ag_snap = os.path.join(out_pth, "ag_snap.csv")
ag_snap = pd.read_csv(ag_snap)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Tue Dec 28 14:07:59 2021


end: Tue Dec 28 14:08:45 2021

--- processing complete


In [11]:
G_flood = nx.read_gpickle("SEN-Cdrive/outputs/gTime_flood10.pickle")

In [12]:
# We only need to find the origin-destination pairs for nodes closest to the origins and services,
# and some nodes will be the nearest for more than one service (and definitely for multiple origins).
list_hamlet = list(hamlet_snap.NN.unique())
list_ag = list(ag_snap.NN.unique())
originslist = list_hamlet + list_ag
origins = list(set(originslist))

In [13]:
dests = list(HDurban_snap.NN.unique()) 

In [14]:
len(origins)

637854

In [15]:
len(dests) 

58

In [16]:
fail_value = 999999999 # If there is no shortest path, the OD pair will be assigned the fail value.

In [17]:
print('start: %s\n' % time.ctime())
OD = gn.calculate_OD(G_flood, origins, dests, fail_value, weight = 't10')
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Tue Dec 28 14:10:29 2021


end: Tue Dec 28 14:30:15 2021

--- processing complete


In [18]:
OD_df = pd.DataFrame(OD, index = origins, columns = dests)

In [19]:
OD_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637854 entries, 3571449893 to 8925478824
Data columns (total 58 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   6058226279  637854 non-null  float64
 1   6029307183  637854 non-null  float64
 2   4998093094  637854 non-null  float64
 3   2201506815  637854 non-null  float64
 4   3474499811  637854 non-null  float64
 5   1697006012  637854 non-null  float64
 6   1901689169  637854 non-null  float64
 7   6032060028  637854 non-null  float64
 8   6040927878  637854 non-null  float64
 9   3449495495  637854 non-null  float64
 10  3990543961  637854 non-null  float64
 11  8972391475  637854 non-null  float64
 12  3418418812  637854 non-null  float64
 13  1983641803  637854 non-null  float64
 14  6014451367  637854 non-null  float64
 15  6027163276  637854 non-null  float64
 16  2833577858  637854 non-null  float64
 17  4656728818  637854 non-null  float64
 18  6045659373  637854 non-null  fl

In [20]:
# Convert to minutes and save to file.
OD_min = OD_df[OD_df <fail_value] / 60
OD_min.to_csv(os.path.join(out_pth, 'OD_flood10_allorigins.csv'))
OD_min

,6058226279,6029307183,4998093094,2201506815,3474499811,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,1936967272,3496518021,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
3571449893,212.584065,72.370016,263.336586,178.755639,161.903071,296.154013,309.228517,293.150349,227.295815,261.330804,...,350.466140,403.469275,155.710545,155.527216,267.669804,306.069012,306.987423,312.310460,259.120605,271.617579
3571449966,212.970977,72.756927,263.723498,179.142551,162.289982,296.540925,309.615428,293.537260,227.682727,261.717716,...,350.853051,403.856187,156.097456,155.914128,268.056716,306.455923,307.374334,312.697372,259.507517,272.004490
3405774993,84.548330,164.723247,253.520017,168.939070,152.086502,286.337444,299.411948,283.333780,217.479246,251.514235,...,340.649571,393.652706,290.908473,290.725144,257.853235,296.252443,297.170854,302.493891,249.304036,261.801010
3405774994,84.949079,164.322498,253.119268,168.538321,151.685753,285.936695,299.011199,282.933031,217.078497,251.113486,...,340.248822,393.251957,290.507724,290.324395,257.452486,295.851694,296.770105,302.093142,248.903287,261.400261
3405774995,85.282625,163.988952,252.785722,168.204775,151.352207,285.603149,298.677653,282.599485,216.744951,250.779940,...,339.915276,392.918411,290.174178,289.990849,257.118940,295.518148,296.436559,301.759596,248.569741,261.066715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8633974649,289.719581,146.027718,436.060128,351.479181,334.626612,468.877555,481.952058,465.873890,400.019357,434.054346,...,523.189681,576.192817,188.633881,188.450553,440.393346,478.792554,479.710964,485.034002,431.844147,444.341121
8633974652,289.629165,145.937301,435.969711,351.388764,334.536196,468.787138,481.861642,465.783474,399.928940,433.963930,...,523.099265,576.102400,188.543465,188.360136,440.302929,478.702137,479.620548,484.943585,431.753731,444.250704
8633974653,289.563206,145.871342,435.903752,351.322805,334.470237,468.721179,481.795683,465.717515,399.862981,433.897970,...,523.033306,576.036441,188.477505,188.294177,440.236970,478.636178,479.554589,484.877626,431.687771,444.184745
8633974654,289.530430,145.838566,435.870977,351.290029,334.437461,468.688404,481.762907,465.684739,399.830206,433.865195,...,523.000530,576.003665,188.444730,188.261401,440.204195,478.603402,479.521813,484.844851,431.654996,444.151969


In [21]:
# Create origin-specific matrix and save to file.
OD_ag = OD_df.loc[list_ag,: ]
OD_ag = OD_ag[OD_ag < fail_value] / 60 
OD_ag.to_csv(os.path.join(out_pth, 'OD_flood10_ag.csv'))
OD_ag

,6058226279,6029307183,4998093094,2201506815,3474499811,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,1936967272,3496518021,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
3507831609,83.755636,232.445763,411.937336,327.356389,310.503821,444.754763,457.829267,441.751099,375.896565,409.931555,...,499.066890,552.070025,427.474675,427.291346,416.270554,454.669762,455.588173,460.911210,407.721356,420.218329
3507831510,90.114132,238.804260,418.295833,333.714886,316.862317,451.113260,464.187763,448.109595,382.255062,416.290051,...,505.425386,558.428522,433.833171,433.649843,422.629051,461.028259,461.946669,467.269707,414.079852,426.576826
6188134127,25.237636,173.927763,353.419336,268.838389,251.985821,386.236763,399.311267,383.233099,317.378565,351.413554,...,440.548890,493.552025,368.956675,368.773346,357.752554,396.151762,397.070173,402.393210,349.203355,361.700329
8631201421,96.422509,176.244632,424.604209,340.023262,323.170694,457.421636,470.496140,454.417972,388.563438,422.598428,...,511.733763,564.736898,367.072725,366.889397,428.937427,467.336635,468.255046,473.578083,420.388229,432.885202
8598305977,53.438771,155.021674,381.620471,297.039524,280.186956,414.437898,427.512402,411.434234,345.579700,379.614690,...,468.750025,521.753160,350.050585,349.867257,385.953689,424.352897,425.271308,430.594345,377.404491,389.901464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651042474,95949.366111,95911.321862,95670.088416,95706.949409,95794.652216,95634.223092,95672.691296,95617.275126,95661.910215,95629.657776,...,95541.272988,95480.310462,95989.024473,95988.666461,95671.605154,95664.516122,95663.626244,95663.519466,95631.759015,95632.765133
3651042508,95941.215857,95903.171607,95661.938161,95698.799154,95786.501962,95626.072838,95664.541042,95609.124871,95653.759960,95621.507521,...,95533.122733,95472.160207,95980.874218,95980.516206,95663.454899,95656.365867,95655.475989,95655.369211,95623.608760,95624.614878
3651042501,95946.710972,95908.666723,95667.433277,95704.294269,95791.997077,95631.567953,95670.036157,95614.619986,95659.255076,95627.002637,...,95538.617849,95477.655323,95986.369334,95986.011322,95668.950015,95661.860983,95660.971105,95660.864327,95629.103876,95630.109994
3651042393,95933.180670,95895.136420,95653.902975,95690.763967,95778.466775,95618.037651,95656.505855,95601.089684,95645.724773,95613.472335,...,95525.087547,95464.125021,95972.839031,95972.481019,95655.419712,95648.330680,95647.440802,95647.334025,95615.573574,95616.579692


In [22]:
OD_hamlet = OD_df.loc[list_hamlet,: ]
OD_hamlet = OD_hamlet[OD_hamlet < fail_value] / 60 
OD_hamlet.to_csv(os.path.join(out_pth, 'OD_flood10_hamlet.csv'))
OD_hamlet

,6058226279,6029307183,4998093094,2201506815,3474499811,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,1936967272,3496518021,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
7761872870,62.891073,245.798986,389.498891,304.917944,288.065376,422.316318,435.390822,419.312654,353.458120,387.493110,...,476.628445,529.631580,426.887347,426.704018,393.832109,432.231317,433.149728,438.472765,385.282911,397.779884
7761872869,62.872513,245.780427,389.480332,304.899385,288.046817,422.297759,435.372262,419.294095,353.439561,387.474550,...,476.609885,529.613021,426.868787,426.685459,393.813550,432.212758,433.131168,438.454206,385.264351,397.761325
6442044321,61.862681,244.770595,388.470500,303.889553,287.036985,421.287927,434.362430,418.284263,352.429729,386.464718,...,475.600053,528.603189,425.858955,425.675627,392.803718,431.202926,432.121336,437.444374,384.254519,396.751493
2142496418,63.316203,246.224117,389.924022,305.343075,288.490507,422.741449,435.815952,419.737785,353.883251,387.918240,...,477.053575,530.056711,427.312477,427.129149,394.257240,432.656448,433.574858,438.897896,385.708041,398.205015
2142496429,63.698577,246.606491,390.306396,305.725449,288.872881,423.123823,436.198326,420.120159,354.265625,388.300614,...,477.435949,530.439085,427.694851,427.511523,394.639614,433.038822,433.957232,439.280270,386.090415,398.587389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9207762346,590.010067,538.743620,392.316477,344.627112,435.296172,372.302310,411.790066,365.139842,297.697339,262.766567,...,284.677977,244.782814,348.428662,348.070650,396.649695,406.848714,406.320451,406.232911,272.057049,265.128454
8463584916,495.475639,457.431389,221.796126,250.092684,340.761744,185.930802,224.399006,168.982836,203.162911,169.766119,...,91.112914,64.552430,505.146696,504.788684,223.312864,219.457655,218.929391,218.841851,171.867358,172.873476
8463593882,499.234844,461.190594,225.764336,253.851889,344.520949,189.899012,228.367216,172.951045,206.922116,173.525324,...,94.872119,68.659633,492.534686,492.176674,227.281073,223.425864,222.897601,222.810061,175.626563,176.632681
9208004175,95942.692657,95904.648407,95663.414961,95700.275954,95787.978761,95627.549637,95666.017842,95610.601671,95655.236760,95622.984321,...,95534.599533,95473.637007,95982.351018,95981.993006,95664.931699,95657.842667,95656.952789,95656.846011,95625.085560,95626.091678


### Filter 1st nearest

#### Check each file to make sure nearest neighbor column is named correctly. If not, rename.

In [16]:
# Reload from file even if already loaded. Quickest way to ensure NN is a column rather than only the index.
OD_hamlet = os.path.join(out_pth, "OD_flood20_hamlet.csv")
OD_hamlet = pd.read_csv(OD_hamlet)

In [12]:
OD_ag = os.path.join(out_pth, "OD_flood10_ag.csv")
OD_ag = pd.read_csv(OD_ag)

In [13]:
OD_ag.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) 
OD_ag

,NN,6058226279,6029307183,4998093094,2201506815,3474499811,1697006012,1901689169,6032060028,6040927878,...,1968458114,1936967272,3496518021,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
0,3507831609,83.755636,232.445763,411.937336,327.356389,310.503821,444.754763,457.829267,441.751099,375.896565,...,499.066890,552.070025,427.474675,427.291346,416.270554,454.669762,455.588173,460.911210,407.721356,420.218329
1,3507831510,90.114132,238.804260,418.295833,333.714886,316.862317,451.113260,464.187763,448.109595,382.255062,...,505.425386,558.428522,433.833171,433.649843,422.629051,461.028259,461.946669,467.269707,414.079852,426.576826
2,6188134127,25.237636,173.927763,353.419336,268.838389,251.985821,386.236763,399.311267,383.233099,317.378565,...,440.548890,493.552025,368.956675,368.773346,357.752554,396.151762,397.070173,402.393210,349.203355,361.700329
3,8631201421,96.422509,176.244632,424.604209,340.023262,323.170694,457.421636,470.496140,454.417972,388.563438,...,511.733763,564.736898,367.072725,366.889397,428.937427,467.336635,468.255046,473.578083,420.388229,432.885202
4,8598305977,53.438771,155.021674,381.620471,297.039524,280.186956,414.437898,427.512402,411.434234,345.579700,...,468.750025,521.753160,350.050585,349.867257,385.953689,424.352897,425.271308,430.594345,377.404491,389.901464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625846,3651042474,95949.366111,95911.321862,95670.088416,95706.949409,95794.652216,95634.223092,95672.691296,95617.275126,95661.910215,...,95541.272988,95480.310462,95989.024473,95988.666461,95671.605154,95664.516122,95663.626244,95663.519466,95631.759015,95632.765133
625847,3651042508,95941.215857,95903.171607,95661.938161,95698.799154,95786.501962,95626.072838,95664.541042,95609.124871,95653.759960,...,95533.122733,95472.160207,95980.874218,95980.516206,95663.454899,95656.365867,95655.475989,95655.369211,95623.608760,95624.614878
625848,3651042501,95946.710972,95908.666723,95667.433277,95704.294269,95791.997077,95631.567953,95670.036157,95614.619986,95659.255076,...,95538.617849,95477.655323,95986.369334,95986.011322,95668.950015,95661.860983,95660.971105,95660.864327,95629.103876,95630.109994
625849,3651042393,95933.180670,95895.136420,95653.902975,95690.763967,95778.466775,95618.037651,95656.505855,95601.089684,95645.724773,...,95525.087547,95464.125021,95972.839031,95972.481019,95655.419712,95648.330680,95647.440802,95647.334025,95615.573574,95616.579692


In [17]:
OD_hamlet.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) 
OD_hamlet

,NN,6058226279,6029307183,4998093094,2201506815,3474499811,1697006012,1901689169,6032060028,6040927878,...,1968458114,1936967272,3496518021,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
0,7761872870,63.403221,874.359225,829.335161,708.179991,291.019814,792.126517,831.614273,788.395031,707.248449,...,706.998875,763.004594,428.940834,428.757505,830.851899,826.672921,826.144657,826.057118,686.470963,685.749430
1,7761872869,63.384661,874.340665,829.316602,708.161432,291.001255,792.107957,831.595713,788.376471,707.229889,...,706.980315,762.986034,428.922274,428.738946,830.833339,826.654362,826.126098,826.038558,686.452404,685.730871
2,6442044321,62.374829,873.330833,828.306770,707.151600,289.991423,791.098125,830.585881,787.366639,706.220057,...,705.970483,761.976202,427.912442,427.729114,829.823507,825.644530,825.116266,825.028726,685.442572,684.721039
3,2142496418,63.828351,874.784355,829.760292,708.605122,291.444945,792.551647,832.039403,788.820161,707.673579,...,707.424005,763.429724,429.365964,429.182636,831.277029,827.098052,826.569788,826.482248,686.896094,686.174561
4,2142496429,64.210725,875.166729,830.142666,708.987496,291.827319,792.934021,832.421777,789.202535,708.055953,...,707.806379,763.812098,429.748338,429.565010,831.659403,827.480426,826.952162,826.864622,687.278468,686.556935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61225,9207762346,759.815291,1270.957620,441.918070,366.535448,671.915052,404.709426,444.197182,391.234121,319.831358,...,305.620498,249.148226,398.206819,398.023490,443.434808,433.572803,432.682925,432.576147,293.401199,286.416565
61226,8463584916,719.145603,1428.350095,230.836629,251.844536,595.818867,194.647536,233.115741,177.606300,205.140446,...,92.201681,64.552430,555.599294,555.415966,232.353367,228.174389,227.646125,227.558585,173.050546,174.056664
61227,8463593882,722.904808,1415.738086,234.804839,255.603741,599.578072,198.615746,237.083950,181.574510,208.899651,...,95.960887,68.659633,542.987285,542.803956,236.321576,232.142599,231.614335,231.526795,176.809751,177.815869
61228,9208004175,163905.632985,164639.432529,163399.717166,163435.116360,163782.306249,163363.528073,163399.830202,163346.486837,163388.412269,...,163265.217738,163203.751207,163766.681729,163766.498400,163401.233904,163388.175784,163387.285906,163387.179128,163356.322369,163357.328487


#### Find first, second, and third nearest destination for each origin node. 

In [15]:
fail_value = 999999999

In [27]:
# Nearest
OD_ag["ag_HD1F10"] = 0
sub = OD_ag.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_ag["ag_HD1F10"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
ag1 = OD_ag[['NN', 'ag_HD1F10']] # Remove unnecessary OD values.


# Second nearest
dupes = OD_ag.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_ag.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.
OD_ag["ag_HD2F10"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_ag["ag_HD2F10"] = Dsub.min(axis=1) 
ag2 = OD_ag.loc[:,['NN', 'ag_HD2F10']] 


# Third nearest
dupes = OD_ag.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_ag.where(~dupes, fail_value)

OD_ag["ag_HD3F10"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_ag["ag_HD3F10"] = Dsub.min(axis=1)
ag3 = OD_ag.loc[:,['NN', 'ag_HD3F10']]

# Combine and write to file
ag_all = OD_ag.loc[:,['NN', 'ag_HD1F10', 'ag_HD2F10', 'ag_HD3F10']]
ag_all.to_csv(os.path.join(out_pth, 'ag_to_HDurban_flood10.csv'))
ag_all.head()

,NN,ag_HD1F10,ag_HD2F10,ag_HD3F10
0,3507831609,82.054823,83.755636,232.445763
1,3507831510,88.413319,90.114132,238.804260
2,6188134127,23.536822,25.237636,173.927763
3,8631201421,94.721696,96.422509,176.244632
4,8598305977,51.737958,53.438771,155.021674


In [18]:
# Nearest
OD_hamlet["ha_HD1F20"] = 0
sub = OD_hamlet.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_hamlet["ha_HD1F20"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
hamlet1 = OD_hamlet[['NN', 'ha_HD1F20']] # Remove unnecessary OD values.


# Second nearest
dupes = OD_hamlet.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_hamlet.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.
OD_hamlet["ha_HD2F20"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_hamlet["ha_HD2F20"] = Dsub.min(axis=1) 
hamlet2 = OD_hamlet.loc[:,['NN', 'ha_HD2F20']] 


# Third nearest
dupes = OD_hamlet.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_hamlet.where(~dupes, fail_value)
OD_hamlet["ha_HD3F20"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_hamlet["ha_HD3F20"] = Dsub.min(axis=1)
hamlet3 = OD_hamlet.loc[:,['NN', 'ha_HD3F20']]


# Combine and write to file
hamlet_all = OD_hamlet.loc[:,['NN', 'ha_HD1F20', 'ha_HD2F20', 'ha_HD3F20']]
hamlet_all.to_csv(os.path.join(out_pth, 'hamlet_to_HDurban_flood20.csv'))
hamlet_all.head()

,NN,ha_HD1F20,ha_HD2F20,ha_HD3F20
0,7761872870,63.403221,66.771529,291.019814
1,7761872869,63.384661,66.752970,291.001255
2,6442044321,62.374829,65.743138,289.991423
3,2142496418,63.828351,67.196660,291.444945
4,2142496429,64.210725,67.579034,291.827319


### Join back to georeferenced _snap file.

In [13]:
# If starting new session, reload from file.
ag_all = os.path.join(out_pth, "ag_to_HDurban_flood20.csv")
ag_all = pd.read_csv(ag_all)
ag_all.info() # Check to make sure NN data type matches its corresponding _snap file

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541461 entries, 0 to 541460
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  541461 non-null  int64  
 1   NN          541461 non-null  int64  
 2   ag_HD1F10   541453 non-null  float64
 3   ag_HD2F10   541461 non-null  float64
 4   ag_HD3F10   541461 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 20.7 MB


In [ ]:
hamlet_all = os.path.join(out_pth, "hamlet_to_HDurban_flood20.csv")
hamlet_all = pd.read_csv(hamlet_all)
hamlet_all.info()

In [10]:
ag_snap = os.path.join(out_pth, "ag_snap.csv")
ag_snap = pd.read_csv(ag_snap)
ag_snap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6852701 entries, 0 to 6852700
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   ID          int64  
 2   LC_90m      int64  
 3   geometry    object 
 4   NN          int64  
 5   NN_dist     float64
dtypes: float64(1), int64(4), object(1)
memory usage: 313.7+ MB


In [ ]:
hamlet_snap = os.path.join(out_pth, "hamlet_snap.csv")
hamlet_snap = pd.read_csv(hamlet_snap)
hamlet_snap.info()

In [29]:
ag_to_HDurban = pd.merge(ag_snap, ag_all, on='NN',how='left')
ag_to_HDurban

,Unnamed: 0,ID_ag,ID_spam,grid_val,ID_LC,val,x,y,NSnomax,geometry,NN,NN_dist,ag_HD1F20,ag_HD2F20,ag_HD3F20
0,0,13941103,2344.0,2008851.0,0.1,2008851.0,-16.458988,12.208646,NaN,POINT (-16.458987999999977 12.208646000000044),3507831609,397.198533,82.054823,83.755636,312.946111
1,1,13941122,2345.0,94605.0,0.1,94605.0,-16.375655,12.208646,NaN,POINT (-16.375654999999938 12.208646000000044),3507831510,2881.578460,88.413319,90.114132,319.304608
2,2,13941017,2346.0,708413.0,0.1,708413.0,-16.208989,12.208646,NaN,POINT (-16.208988999999974 12.208646000000044),6188134127,17711.673781,23.536822,25.237636,254.428111
3,3,13941036,2402.0,315719.0,0.1,315719.0,-16.208492,12.152815,NaN,POINT (-16.20849199999998 12.152815000000032),3507831510,21562.215777,88.413319,90.114132,319.304608
4,4,13941044,2401.0,375288.0,0.1,375288.0,-16.294213,12.149487,NaN,POINT (-16.294212999999957 12.149487000000022),3507831510,12874.077085,88.413319,90.114132,319.304608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13941119,13941119,13941081,3.0,87412.0,0.1,87412.0,-15.291536,16.874047,NaN,POINT (-15.291535999999951 16.874047000000076),3651042474,2658.421843,139785.574025,139786.528309,139792.211856
13941120,13941120,13941093,6.0,21660.0,0.1,21660.0,-15.292326,16.791961,NaN,POINT (-15.292325999999946 16.79196100000007),3651042508,1234.146014,139777.423770,139778.378054,139784.061601
13941121,13941121,13941109,2.0,70993.0,0.1,70993.0,-15.374255,16.867753,NaN,POINT (-15.374254999999948 16.86775300000005),3651042501,9172.299324,139782.918886,139783.873169,139789.556716
13941122,13941122,13941119,5.0,54822.0,0.1,54822.0,-15.375659,16.791961,NaN,POINT (-15.375658999999928 16.79196100000007),3651042393,4280.425149,139769.388584,139770.342867,139776.026414


In [30]:
hamlet_to_HDurban = pd.merge(hamlet_snap, hamlet_all, on='NN',how='left')
hamlet_to_HDurban

,Unnamed: 0,Unnamed_ 0,mgrs_code,type,GlobalID,Shape_Leng,Shape_Area,geometry,NN,NN_dist,ha_HD1F20,ha_HD2F20,ha_HD3F20
0,0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,POINT (-16.721473282454415 12.348636090165247),7761872870,307.058089,63.403221,66.771529,291.019814
1,1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,POINT (-16.716456507935607 12.34788723564788),7761872869,801.450257,63.384661,66.752970,291.001255
2,2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,POINT (-16.713855008830738 12.350880992129111),6442044321,694.273717,62.374829,65.743138,289.991423
3,3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,POINT (-16.701275174874546 12.355585269999269),2142496418,689.246791,63.828351,67.196660,291.444945
4,4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,POINT (-16.698773736706396 12.356804484409668),2142496429,607.912599,64.210725,67.579034,291.827319
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125881,125881,125881,28QED6412_03,hamlet,{5555A010-36B2-47D2-96C4-BDD1E59111ED},0.005397,0.000002,POINT (-14.397827933065358 16.394142941310925),8592243241,5089.086102,191.325867,193.414105,199.472924
125882,125882,125882,28QED6413_03,hamlet,{20205A44-8B9D-4FCE-B14C-53826594DB5A},0.003610,0.000001,POINT (-14.397473236932905 16.405676938062538),6375187769,3949.180081,189.396436,191.484673,197.543492
125883,125883,125883,28QED6413_04,hamlet,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},0.015471,0.000008,POINT (-14.400364192239715 16.404303134272737),8592243457,4008.851333,191.027109,193.115346,199.174166
125884,125884,125884,28QED6424_03,hamlet,{51593C65-B268-4BA1-8212-E43232C021FF},0.003883,0.000001,POINT (-14.396243884215123 16.49715066208162),3646207611,1826.174300,187.250035,189.338272,195.397091


In [31]:
hamlet_to_HDurban.to_csv(os.path.join(out_pth, 'hamlet_to_HDurban_flood20.csv'))
ag_to_HDurban.to_csv(os.path.join(out_pth, 'ag_to_HDurban_flood20.csv'))

### Combine with cost-distance raster travel times from origins to road node.

In [8]:
# If reloading
ag_to_HDurban = os.path.join(out_pth, "ag_to_HDurban_flood20.csv")
ag_to_HDurban = pd.read_csv(ag_to_HDurban)
ag_to_HDurban.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 16 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   Unnamed: 0.1  int64  
 2   ID_ag         int64  
 3   ID_spam       float64
 4   grid_val      float64
 5   ID_LC         float64
 6   val           float64
 7   x             float64
 8   y             float64
 9   NSnomax       float64
 10  geometry      object 
 11  NN            int64  
 12  NN_dist       float64
 13  ag_HD1F20     float64
 14  ag_HD2F20     float64
 15  ag_HD3F20     float64
dtypes: float64(11), int64(4), object(1)
memory usage: 1.7+ GB


In [9]:
ag_to_HDurban['NSnomax'].replace({-9999: np.nan},inplace =True)
ag_to_HDurban

,Unnamed: 0,Unnamed: 0.1,ID_ag,ID_spam,grid_val,ID_LC,val,x,y,NSnomax,geometry,NN,NN_dist,ag_HD1F20,ag_HD2F20,ag_HD3F20
0,0,0,13941103,2344.0,2008851.0,0.1,2008851.0,-16.458988,12.208646,NaN,POINT (-16.458987999999977 12.208646000000044),3507831609,397.198533,82.054823,83.755636,312.946111
1,1,1,13941122,2345.0,94605.0,0.1,94605.0,-16.375655,12.208646,NaN,POINT (-16.375654999999938 12.208646000000044),3507831510,2881.578460,88.413319,90.114132,319.304608
2,2,2,13941017,2346.0,708413.0,0.1,708413.0,-16.208989,12.208646,NaN,POINT (-16.208988999999974 12.208646000000044),6188134127,17711.673781,23.536822,25.237636,254.428111
3,3,3,13941036,2402.0,315719.0,0.1,315719.0,-16.208492,12.152815,NaN,POINT (-16.20849199999998 12.152815000000032),3507831510,21562.215777,88.413319,90.114132,319.304608
4,4,4,13941044,2401.0,375288.0,0.1,375288.0,-16.294213,12.149487,NaN,POINT (-16.294212999999957 12.149487000000022),3507831510,12874.077085,88.413319,90.114132,319.304608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13941119,13941119,13941119,13941081,3.0,87412.0,0.1,87412.0,-15.291536,16.874047,NaN,POINT (-15.291535999999951 16.874047000000076),3651042474,2658.421843,139785.574025,139786.528309,139792.211856
13941120,13941120,13941120,13941093,6.0,21660.0,0.1,21660.0,-15.292326,16.791961,NaN,POINT (-15.292325999999946 16.79196100000007),3651042508,1234.146014,139777.423770,139778.378054,139784.061601
13941121,13941121,13941121,13941109,2.0,70993.0,0.1,70993.0,-15.374255,16.867753,NaN,POINT (-15.374254999999948 16.86775300000005),3651042501,9172.299324,139782.918886,139783.873169,139789.556716
13941122,13941122,13941122,13941119,5.0,54822.0,0.1,54822.0,-15.375659,16.791961,NaN,POINT (-15.375658999999928 16.79196100000007),3651042393,4280.425149,139769.388584,139770.342867,139776.026414


In [10]:
ag_to_HDurban['HD1mmF20'] = 0 # mm for multi-modal
ag_to_HDurban['HD1mmF20'] = ag_to_HDurban['ag_HD1F20'] + ag_to_HDurban['NSnomax']

In [11]:
# Remove unnecessary data. These data are still saved in the _to_HDurban files.
ag_to_HDurban = ag_to_HDurban[['ID_ag', 'val', 'x', 'y', 'NSnomax', 'NN', 'HD1mmF20']]
ag_to_HDurban

,ID_ag,val,x,y,NSnomax,NN,HD1mmF20
0,13941103,2008851.0,-16.458988,12.208646,NaN,3507831609,NaN
1,13941122,94605.0,-16.375655,12.208646,NaN,3507831510,NaN
2,13941017,708413.0,-16.208989,12.208646,NaN,6188134127,NaN
3,13941036,315719.0,-16.208492,12.152815,NaN,3507831510,NaN
4,13941044,375288.0,-16.294213,12.149487,NaN,3507831510,NaN
...,...,...,...,...,...,...,...
13941119,13941081,87412.0,-15.291536,16.874047,NaN,3651042474,NaN
13941120,13941093,21660.0,-15.292326,16.791961,NaN,3651042508,NaN
13941121,13941109,70993.0,-15.374255,16.867753,NaN,3651042501,NaN
13941122,13941119,54822.0,-15.375659,16.791961,NaN,3651042393,NaN


In [12]:
print(len(ag_to_HDurban.loc[ag_to_HDurban['HD1mmF20']>240]), end="\n") # Number of origins isolated by excessive travel time
print(len(ag_to_HDurban.loc[ag_to_HDurban['HD1mmF20'].isnull()])) # Number of origins isolated by inability to access road
# Second value should match the non-flooded results (755160), as the cost-distance raster does not incorporate the flood penalties.

1163857
755160


In [29]:
# Add non-flooded multi-modal values to table.
ag_base = os.path.join(out_pth, "temp_fromag_drive-CD.csv")
ag_base = pd.read_csv(ag_base)
ag_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 5 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ID_ag     int64  
 1   val       float64
 2   NSnomax   float64
 3   NN        int64  
 4   ag_HD1mm  float64
dtypes: float64(3), int64(2)
memory usage: 531.8 MB


In [37]:
ag_base = ag_base[['ID_ag', 'ag_HD1mm']]
ag_base.rename(columns={'ag_HD1mm': 'HD1mm'}, inplace=True)
ag_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ID_ag   int64  
 1   HD1mm   float64
dtypes: float64(1), int64(1)
memory usage: 212.7 MB


In [38]:
ag_to_HDurban = pd.merge(ag_to_HDurban, ag_base, on='ID_ag',how='left')
ag_to_HDurban.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941124 entries, 0 to 13941123
Data columns (total 8 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ID_ag     int64  
 1   val       float64
 2   x         float64
 3   y         float64
 4   NSnomax   float64
 5   NN        int64  
 6   HD1mmF20  float64
 7   HD1mm     float64
dtypes: float64(6), int64(2)
memory usage: 957.3 MB


In [39]:
ag_to_HDurban['HD1dif'] = 0 # dif for difference between the two travel times (in minutes)
ag_to_HDurban['HD1pc'] = 0 # pc for percent change
ag_to_HDurban['HD1dif'] = ag_to_HDurban['HD1mmF20'] - ag_to_HDurban['HD1mm'] # Travel time is X minutes longer in flood conditions.
ag_to_HDurban['HD1pc'] = ag_to_HDurban['HD1dif'] / ag_to_HDurban['HD1mm'] * 100 # Travel time is X percent longer in flood conditions.
ag_to_HDurban

,ID_ag,val,x,y,NSnomax,NN,HD1mmF20,HD1mm,HD1dif,HD1pc
0,13941103,2008851.0,-16.458988,12.208646,NaN,3507831609,NaN,NaN,NaN,NaN
1,13941122,94605.0,-16.375655,12.208646,NaN,3507831510,NaN,NaN,NaN,NaN
2,13941017,708413.0,-16.208989,12.208646,NaN,6188134127,NaN,NaN,NaN,NaN
3,13941036,315719.0,-16.208492,12.152815,NaN,3507831510,NaN,NaN,NaN,NaN
4,13941044,375288.0,-16.294213,12.149487,NaN,3507831510,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
13941119,13941081,87412.0,-15.291536,16.874047,NaN,3651042474,NaN,NaN,NaN,NaN
13941120,13941093,21660.0,-15.292326,16.791961,NaN,3651042508,NaN,NaN,NaN,NaN
13941121,13941109,70993.0,-15.374255,16.867753,NaN,3651042501,NaN,NaN,NaN,NaN
13941122,13941119,54822.0,-15.375659,16.791961,NaN,3651042393,NaN,NaN,NaN,NaN


In [40]:
print(len(ag_to_HDurban.loc[ag_to_HDurban['HD1mmF20']>240]), end="\n") # Number of origins isolated by excessive travel time
print(len(ag_to_HDurban.loc[ag_to_HDurban['HD1mmF20'].isnull()])) # Number of origins isolated by inability to access road

# Second value should match the non-flooded results, as the cost-distance raster does not incorporate the flood penalties.
print(len(ag_to_HDurban.loc[ag_to_HDurban['HD1mm']>240]), end="\n") # Number of origins isolated by excessive travel time
print(len(ag_to_HDurban.loc[ag_to_HDurban['HD1mm'].isnull()]))

1163857
755160
50507
755160


In [42]:
display(ag_to_HDurban.describe())

,ID_ag,val,x,y,NSnomax,NN,HD1mmF20,HD1mm,HD1dif,HD1pc
count,1.394112e+07,1.394112e+07,1.394112e+07,1.394112e+07,1.318626e+07,1.394112e+07,1.318596e+07,1.318596e+07,1.318596e+07,1.318596e+07
mean,6.970562e+06,1.219702e+02,-1.558418e+01,1.432788e+01,2.209557e+01,5.121305e+09,5.217945e+02,7.234398e+01,4.494505e+02,3.929297e+02
std,4.024456e+06,3.760062e+03,9.437674e-01,9.172257e-01,2.796064e+01,2.262019e+09,5.061590e+03,4.196269e+01,5.052823e+03,2.803781e+03
min,0.000000e+00,1.563126e-01,-1.748885e+01,1.204963e+01,0.000000e+00,2.214129e+07,0.000000e+00,0.000000e+00,-1.136868e-13,-1.000430e-13
25%,3.485281e+06,4.293236e+01,-1.622061e+01,1.368601e+01,5.456861e+00,3.656862e+09,4.999677e+01,4.336144e+01,4.113212e-01,6.045770e-01
50%,6.970562e+06,7.008440e+01,-1.578565e+01,1.436783e+01,1.162371e+01,4.519739e+09,8.137713e+01,6.605466e+01,2.693749e+00,4.421227e+00
75%,1.045584e+07,1.013998e+02,-1.516850e+01,1.491868e+01,2.869398e+01,6.968809e+09,1.274517e+02,9.401759e+01,3.236852e+01,3.900011e+01
max,1.394112e+07,6.939670e+06,-1.115399e+01,1.687405e+01,1.713278e+03,9.285641e+09,2.218282e+05,1.801212e+03,2.213788e+05,2.456662e+05


In [45]:
print(ag_to_HDurban["HD1mmF20"].mean())
print(ag_to_HDurban["HD1mm"].mean())

521.7944964677994
72.343983276716


In [48]:
ag_to_HDurban.to_csv(os.path.join(out_pth, 'ag_to_HDurban_mmF20.csv'))

#### Hamlets

In [14]:
hamlet_to_HDurban = os.path.join(out_pth, "hamlet_to_HDurban_flood20.csv")
hamlet_to_HDurban = pd.read_csv(hamlet_to_HDurban)
hamlet_to_HDurban.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125886 entries, 0 to 125885
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    125886 non-null  int64  
 1   Unnamed: 0.1  125886 non-null  int64  
 2   Unnamed_ 0    125886 non-null  int64  
 3   mgrs_code     125886 non-null  object 
 4   type          125886 non-null  object 
 5   GlobalID      125886 non-null  object 
 6   Shape_Leng    125886 non-null  float64
 7   Shape_Area    125886 non-null  float64
 8   geometry      125886 non-null  object 
 9   NN            125886 non-null  int64  
 10  NN_dist       125886 non-null  float64
 11  ha_HD1F20     125883 non-null  float64
 12  ha_HD2F20     125886 non-null  float64
 13  ha_HD3F20     125886 non-null  float64
dtypes: float64(6), int64(4), object(4)
memory usage: 13.4+ MB


In [16]:
# Hamlet file didn't already have the travel values from origin to road node. Merging now.
costdist = os.path.join(out_pth, "temp_fromhamlet_drive.csv")
costdist = pd.read_csv(costdist)
costdist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125886 entries, 0 to 125885
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   mgrs_code  125886 non-null  object 
 1   GlobalID   125886 non-null  object 
 2   HD1        125886 non-null  float64
 3   NSnomax    125552 non-null  float64
 4   HD1mm      125552 non-null  float64
dtypes: float64(3), object(2)
memory usage: 4.8+ MB


In [17]:
# Make sure the ID is unique and fully matched
list1 = list(hamlet_to_HDurban['mgrs_code'].unique())
list2 = list(costdist['mgrs_code'].unique())
print(len(list1))
print(len(list2))

125886
125886


In [18]:
hamlet_to_HDurban = pd.merge(hamlet_to_HDurban, costdist, on='mgrs_code',how='left')
hamlet_to_HDurban

,Unnamed: 0,Unnamed: 0.1,Unnamed_ 0,mgrs_code,type,GlobalID_x,Shape_Leng,Shape_Area,geometry,NN,NN_dist,ha_HD1F20,ha_HD2F20,ha_HD3F20,GlobalID_y,HD1,NSnomax,HD1mm
0,0,0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,POINT (-16.721473282454415 12.348636090165247),7761872870,307.058089,63.403221,66.771529,291.019814,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},62.539942,NaN,NaN
1,1,1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,POINT (-16.716456507935607 12.34788723564788),7761872869,801.450257,63.384661,66.752970,291.001255,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},62.521382,NaN,NaN
2,2,2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,POINT (-16.713855008830738 12.350880992129111),6442044321,694.273717,62.374829,65.743138,289.991423,{D03C2B85-5F35-4EE8-8346-B83494628F26},61.511550,NaN,NaN
3,3,3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,POINT (-16.701275174874546 12.355585269999269),2142496418,689.246791,63.828351,67.196660,291.444945,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},62.965072,NaN,NaN
4,4,4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,POINT (-16.698773736706396 12.356804484409668),2142496429,607.912599,64.210725,67.579034,291.827319,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},63.347446,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125881,125881,125881,125881,28QED6412_03,hamlet,{5555A010-36B2-47D2-96C4-BDD1E59111ED},0.005397,0.000002,POINT (-14.397827933065358 16.394142941310925),8592243241,5089.086102,191.325867,193.414105,199.472924,{5555A010-36B2-47D2-96C4-BDD1E59111ED},184.373909,64.9628,249.336709
125882,125882,125882,125882,28QED6413_03,hamlet,{20205A44-8B9D-4FCE-B14C-53826594DB5A},0.003610,0.000001,POINT (-14.397473236932905 16.405676938062538),6375187769,3949.180081,189.396436,191.484673,197.543492,{20205A44-8B9D-4FCE-B14C-53826594DB5A},182.444477,83.4117,265.856177
125883,125883,125883,125883,28QED6413_04,hamlet,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},0.015471,0.000008,POINT (-14.400364192239715 16.404303134272737),8592243457,4008.851333,191.027109,193.115346,199.174166,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},184.075151,86.7114,270.786551
125884,125884,125884,125884,28QED6424_03,hamlet,{51593C65-B268-4BA1-8212-E43232C021FF},0.003883,0.000001,POINT (-14.396243884215123 16.49715066208162),3646207611,1826.174300,187.250035,189.338272,195.397091,{51593C65-B268-4BA1-8212-E43232C021FF},180.281911,56.9831,237.265011


In [19]:
hamlet_to_HDurban['NSnomax'].replace({-9999: np.nan},inplace =True)
hamlet_to_HDurban

,Unnamed: 0,Unnamed: 0.1,Unnamed_ 0,mgrs_code,type,GlobalID_x,Shape_Leng,Shape_Area,geometry,NN,NN_dist,ha_HD1F20,ha_HD2F20,ha_HD3F20,GlobalID_y,HD1,NSnomax,HD1mm
0,0,0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,POINT (-16.721473282454415 12.348636090165247),7761872870,307.058089,63.403221,66.771529,291.019814,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},62.539942,NaN,NaN
1,1,1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,POINT (-16.716456507935607 12.34788723564788),7761872869,801.450257,63.384661,66.752970,291.001255,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},62.521382,NaN,NaN
2,2,2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,POINT (-16.713855008830738 12.350880992129111),6442044321,694.273717,62.374829,65.743138,289.991423,{D03C2B85-5F35-4EE8-8346-B83494628F26},61.511550,NaN,NaN
3,3,3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,POINT (-16.701275174874546 12.355585269999269),2142496418,689.246791,63.828351,67.196660,291.444945,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},62.965072,NaN,NaN
4,4,4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,POINT (-16.698773736706396 12.356804484409668),2142496429,607.912599,64.210725,67.579034,291.827319,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},63.347446,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125881,125881,125881,125881,28QED6412_03,hamlet,{5555A010-36B2-47D2-96C4-BDD1E59111ED},0.005397,0.000002,POINT (-14.397827933065358 16.394142941310925),8592243241,5089.086102,191.325867,193.414105,199.472924,{5555A010-36B2-47D2-96C4-BDD1E59111ED},184.373909,64.9628,249.336709
125882,125882,125882,125882,28QED6413_03,hamlet,{20205A44-8B9D-4FCE-B14C-53826594DB5A},0.003610,0.000001,POINT (-14.397473236932905 16.405676938062538),6375187769,3949.180081,189.396436,191.484673,197.543492,{20205A44-8B9D-4FCE-B14C-53826594DB5A},182.444477,83.4117,265.856177
125883,125883,125883,125883,28QED6413_04,hamlet,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},0.015471,0.000008,POINT (-14.400364192239715 16.404303134272737),8592243457,4008.851333,191.027109,193.115346,199.174166,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},184.075151,86.7114,270.786551
125884,125884,125884,125884,28QED6424_03,hamlet,{51593C65-B268-4BA1-8212-E43232C021FF},0.003883,0.000001,POINT (-14.396243884215123 16.49715066208162),3646207611,1826.174300,187.250035,189.338272,195.397091,{51593C65-B268-4BA1-8212-E43232C021FF},180.281911,56.9831,237.265011


In [20]:
hamlet_to_HDurban['HD1mmF20'] = 0 # mm for multi-modal
hamlet_to_HDurban['HD1mmF20'] = hamlet_to_HDurban['ha_HD1F20'] + hamlet_to_HDurban['NSnomax']

The geometry column is missing a comma for some reason and isn't reading as a GDF in Python or Arc. So let's quick make a shapefile version by extracting the points. Can use this as well for the non-flood travel times, which were saved as csv.

In [21]:
hamlet_to_HDurban["geometry"] = hamlet_to_HDurban["geometry"].astype('str')
hamlet_to_HDurban["geometry"]  = hamlet_to_HDurban["geometry"] .str.strip('POINT ')
hamlet_to_HDurban["geometry"]  = hamlet_to_HDurban["geometry"] .str.strip('()')
XY = hamlet_to_HDurban["geometry"] .str.split(" ", expand=True)
hamlet_to_HDurban["X"] = XY[0]
hamlet_to_HDurban["Y"] = XY[1]
hamlet_to_HDurban["X"] = hamlet_to_HDurban["X"].astype('float')
hamlet_to_HDurban["Y"] = hamlet_to_HDurban["Y"].astype('float')
hamlet_to_HDurban = hamlet_to_HDurban.drop(columns=['geometry'])
hamlet_to_HDurban.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    125886 non-null  int64  
 1   Unnamed: 0.1  125886 non-null  int64  
 2   Unnamed_ 0    125886 non-null  int64  
 3   mgrs_code     125886 non-null  object 
 4   type          125886 non-null  object 
 5   GlobalID_x    125886 non-null  object 
 6   Shape_Leng    125886 non-null  float64
 7   Shape_Area    125886 non-null  float64
 8   NN            125886 non-null  int64  
 9   NN_dist       125886 non-null  float64
 10  ha_HD1F20     125883 non-null  float64
 11  ha_HD2F20     125886 non-null  float64
 12  ha_HD3F20     125886 non-null  float64
 13  GlobalID_y    125886 non-null  object 
 14  HD1           125886 non-null  float64
 15  NSnomax       125552 non-null  float64
 16  HD1mm         125552 non-null  float64
 17  HD1mmF20      125549 non-null  float64
 18  X   

In [22]:
# Remove unnecessary data. These data are still saved in the _to_HDurban files.
hamlet_to_HDurban = hamlet_to_HDurban[['mgrs_code', 'NSnomax', 'NN', 'HD1mm','HD1mmF20', 'X', 'Y']]
hamlet_to_HDurban

,mgrs_code,NSnomax,NN,HD1mm,HD1mmF20,X,Y
0,28PCU1265_01,NaN,7761872870,NaN,NaN,-16.721473,12.348636
1,28PCU1365_01,NaN,7761872869,NaN,NaN,-16.716457,12.347887
2,28PCU1365_02,NaN,6442044321,NaN,NaN,-16.713855,12.350881
3,28PCU1566_01,NaN,2142496418,NaN,NaN,-16.701275,12.355585
4,28PCU1566_02,NaN,2142496429,NaN,NaN,-16.698774,12.356804
...,...,...,...,...,...,...,...
125881,28QED6412_03,64.9628,8592243241,249.336709,256.288667,-14.397828,16.394143
125882,28QED6413_03,83.4117,6375187769,265.856177,272.808136,-14.397473,16.405677
125883,28QED6413_04,86.7114,8592243457,270.786551,277.738509,-14.400364,16.404303
125884,28QED6424_03,56.9831,3646207611,237.265011,244.233135,-14.396244,16.497151


In [24]:
print(len(hamlet_to_HDurban.loc[hamlet_to_HDurban['HD1mmF20']>240]), end="\n") # Number of origins isolated by excessive travel time
print(len(hamlet_to_HDurban.loc[hamlet_to_HDurban['HD1mmF20'].isnull()])) # Number of origins isolated by inability to access road

# Second value should match the non-flooded results, as the cost-distance raster does not incorporate the flood penalties.
print(len(hamlet_to_HDurban.loc[hamlet_to_HDurban['HD1mm']>240]), end="\n") # Number of origins isolated by excessive travel time
print(len(hamlet_to_HDurban.loc[hamlet_to_HDurban['HD1mm'].isnull()]))

26039
337
6541
334


In [27]:
hamlet_to_HDurban['HD1dif'] = 0 # dif for difference between the two travel times (in minutes)
hamlet_to_HDurban['HD1pc'] = 0 # pc for percent change
hamlet_to_HDurban['HD1dif'] = hamlet_to_HDurban['HD1mmF20'] - hamlet_to_HDurban['HD1mm'] # Travel time is X minutes longer in flood conditions.
hamlet_to_HDurban['HD1pc'] = hamlet_to_HDurban['HD1dif'] / hamlet_to_HDurban['HD1mm'] * 100 # Travel time is X percent longer in flood conditions.
hamlet_to_HDurban

,mgrs_code,NSnomax,NN,HD1mm,HD1mmF20,X,Y,geometry,HD1dif,HD1pc
0,28PCU1265_01,NaN,7761872870,NaN,NaN,-16.721473,12.348636,POINT (-16.72147 12.34864),NaN,NaN
1,28PCU1365_01,NaN,7761872869,NaN,NaN,-16.716457,12.347887,POINT (-16.71646 12.34789),NaN,NaN
2,28PCU1365_02,NaN,6442044321,NaN,NaN,-16.713855,12.350881,POINT (-16.71386 12.35088),NaN,NaN
3,28PCU1566_01,NaN,2142496418,NaN,NaN,-16.701275,12.355585,POINT (-16.70128 12.35559),NaN,NaN
4,28PCU1566_02,NaN,2142496429,NaN,NaN,-16.698774,12.356804,POINT (-16.69877 12.35680),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
125881,28QED6412_03,64.9628,8592243241,249.336709,256.288667,-14.397828,16.394143,POINT (-14.39783 16.39414),6.951959,2.788181
125882,28QED6413_03,83.4117,6375187769,265.856177,272.808136,-14.397473,16.405677,POINT (-14.39747 16.40568),6.951958,2.614932
125883,28QED6413_04,86.7114,8592243457,270.786551,277.738509,-14.400364,16.404303,POINT (-14.40036 16.40430),6.951958,2.567320
125884,28QED6424_03,56.9831,3646207611,237.265011,244.233135,-14.396244,16.497151,POINT (-14.39624 16.49715),6.968124,2.936853


In [ ]:
display(hamlet_to_HDurban.describe())

In [28]:
print('start: %s\n' % time.ctime())
crs = "EPSG:4326"
geometry = [Point(xy) for xy in zip(hamlet_to_HDurban.X, hamlet_to_HDurban.Y)]
hamlet_to_HDurban = GeoDataFrame(hamlet_to_HDurban, crs=crs, geometry=geometry) 
hamlet_to_HDurban.to_file(driver='ESRI Shapefile', filename='R:/SEN/GEO/Team/Projects/Sen_TransportOV/hamlet_to_HDurban_mmF20.shp') 
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Mon Dec 27 23:02:45 2021


end: Mon Dec 27 23:03:28 2021

--- processing complete


In [ ]:
hamlet_to_HDurban.to_csv(os.path.join(out_pth, 'hamlet_to_HDurban_mmF20.csv'))

### Add 1 in 50-year event to the base and flood table.

In [ ]:
# If reloading:
agF20 = os.path.join(out_pth, "ag_to_HDurban_mmF20.csv")
agF20 = pd.read_csv(agF20)

In [15]:
agF50 = os.path.join(out_pth, "ag_to_HDurban_flood50.csv")
agF50 = pd.read_csv(agF50)

print(agF20.info())
print(agF50.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 12 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   ID_ag       int64  
 2   val         float64
 3   x           float64
 4   y           float64
 5   NSnomax     float64
 6   NN          int64  
 7   HD1mmF20    float64
 8   HD1mm       float64
 9   HD1dif      float64
 10  HD1pc       float64
 11  geometry    object 
dtypes: float64(8), int64(3), object(1)
memory usage: 1.2+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625851 entries, 0 to 625850
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  625851 non-null  int64  
 1   NN          625851 non-null  int64  
 2   ag_HD1F50   625841 non-null  float64
 3   ag_HD2F50   625851 non-null  float64
 4   ag_HD3F50   625851 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 23.9 MB
None


In [16]:
# Make sure the ID is unique and fully matched
list1 = list(agF50['NN'].unique())
list2 = list(agF20['NN'].unique())
list3 = list(agF10['NN'].unique())
print(len(list1))
print(len(list2))
print(len(list3))

625851
625851


In [17]:
agF50 = agF50[['NN', 'ag_HD1F50']]
agF50

,NN,ag_HD1F50
0,3507831609,82.731533
1,3507831510,89.090029
2,6188134127,23.536822
3,8631201421,503.912228
4,8598305977,518.180056
...,...,...
625846,3651042474,186216.753175
625847,3651042508,186208.602920
625848,3651042501,186214.098036
625849,3651042393,186200.567733


In [18]:
agF50 = pd.merge(agF20, agF50, on='NN',how='left')
agF50

,Unnamed: 0,ID_ag,val,x,y,NSnomax,NN,HD1mmF20,HD1mm,HD1dif,HD1pc,geometry,ag_HD1F50
0,0,13941103,2008851.0,-16.458988,12.208646,NaN,3507831609,NaN,NaN,NaN,NaN,POINT (-16.458988 12.208646),82.731533
1,1,13941122,94605.0,-16.375655,12.208646,NaN,3507831510,NaN,NaN,NaN,NaN,POINT (-16.375655 12.208646),89.090029
2,2,13941017,708413.0,-16.208989,12.208646,NaN,6188134127,NaN,NaN,NaN,NaN,POINT (-16.208989 12.208646),23.536822
3,3,13941036,315719.0,-16.208492,12.152815,NaN,3507831510,NaN,NaN,NaN,NaN,POINT (-16.208492 12.152815),89.090029
4,4,13941044,375288.0,-16.294213,12.149487,NaN,3507831510,NaN,NaN,NaN,NaN,POINT (-16.294213 12.149487),89.090029
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13941119,13941119,13941081,87412.0,-15.291536,16.874047,NaN,3651042474,NaN,NaN,NaN,NaN,POINT (-15.291536 16.874047),186216.753175
13941120,13941120,13941093,21660.0,-15.292326,16.791961,NaN,3651042508,NaN,NaN,NaN,NaN,POINT (-15.292326 16.791961),186208.602920
13941121,13941121,13941109,70993.0,-15.374255,16.867753,NaN,3651042501,NaN,NaN,NaN,NaN,POINT (-15.374255 16.867753),186214.098036
13941122,13941122,13941119,54822.0,-15.375659,16.791961,NaN,3651042393,NaN,NaN,NaN,NaN,POINT (-15.375659 16.791961),186200.567733


In [19]:
agF50['HD1mmF50'] = 0 # mm for multi-modal
agF50['HD1mmF50'] = agF50['ag_HD1F50'] + agF50['NSnomax']

In [20]:
# Rename and remove fields for clarity
agF50.rename(columns={'HD1mmF20':'F20', 'HD1mm':'base', 'HD1dif':'dif_20b', 'HD1pc':'pc_20b', 'HD1mmF50':'F50'}, inplace=True)
agF50.info()
# base: travel times before project intervention, no flood event.
# F20: travel times before project intervention, during 1 in 20-year flood event.
# F50: travel times before project intervention, during 1 in 50-year flood event.
# dif_20b: change in travel time from 'base' travel to a 1 in 20-year flood event. Positive number = travel time increased.
# pc_20b: percent change in travel time from 'base' travel to a 1 in 20-year flood event. Positive number = travel time increased.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941124 entries, 0 to 13941123
Data columns (total 14 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   ID_ag       int64  
 2   val         float64
 3   x           float64
 4   y           float64
 5   NSnomax     float64
 6   NN          int64  
 7   F20         float64
 8   base        float64
 9   dif_20b     float64
 10  pc_20b      float64
 11  geometry    object 
 12  ag_HD1F50   float64
 13  F50         float64
dtypes: float64(10), int64(3), object(1)
memory usage: 1.6+ GB


In [21]:
# Dif and pc
agF50['dif_50b'] = 0 # dif for difference between the two travel times (in minutes)
agF50['pc_50b'] = 0 # pc for percent change
agF50['dif_50b'] = agF50['F50'] - agF50['base'] # Travel time is X minutes longer in flood conditions.
agF50['pc_50b'] = agF50['dif_50b'] / agF50['base'] * 100 # Travel time is X percent longer in flood conditions.
agF50.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941124 entries, 0 to 13941123
Data columns (total 16 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   ID_ag       int64  
 2   val         float64
 3   x           float64
 4   y           float64
 5   NSnomax     float64
 6   NN          int64  
 7   F20         float64
 8   base        float64
 9   dif_20b     float64
 10  pc_20b      float64
 11  geometry    object 
 12  ag_HD1F50   float64
 13  F50         float64
 14  dif_50b     float64
 15  pc_50b      float64
dtypes: float64(12), int64(3), object(1)
memory usage: 1.8+ GB


In [23]:
display(agF50.describe())

,Unnamed: 0,ID_ag,val,x,y,NSnomax,NN,F20,base,dif_20b,pc_20b,ag_HD1F50,F50,dif_50b,pc_50b
count,1.394112e+07,1.394112e+07,1.394112e+07,1.394112e+07,1.394112e+07,1.318626e+07,1.394112e+07,1.318596e+07,1.318596e+07,1.318596e+07,1.318596e+07,1.394082e+07,1.318596e+07,1.318596e+07,1.318596e+07
mean,6.970562e+06,6.970562e+06,1.219702e+02,-1.558418e+01,1.432788e+01,2.209557e+01,5.121305e+09,5.217945e+02,7.234398e+01,4.494505e+02,3.929297e+02,2.455770e+03,1.177547e+03,1.105203e+03,1.143876e+03
std,4.024456e+06,4.024456e+06,3.760062e+03,9.437674e-01,9.172257e-01,2.796064e+01,2.262019e+09,5.061590e+03,4.196269e+01,5.052823e+03,2.803781e+03,1.741179e+04,8.076292e+03,8.066331e+03,6.148201e+03
min,0.000000e+00,0.000000e+00,1.563126e-01,-1.748885e+01,1.204963e+01,0.000000e+00,2.214129e+07,0.000000e+00,0.000000e+00,-1.136868e-13,-1.000430e-13,0.000000e+00,0.000000e+00,-1.705303e-13,-1.000430e-13
25%,3.485281e+06,3.485281e+06,4.293236e+01,-1.622061e+01,1.368601e+01,5.456861e+00,3.656862e+09,4.999677e+01,4.336144e+01,4.113212e-01,6.045770e-01,3.473214e+01,5.288514e+01,8.248076e-01,1.449902e+00
50%,6.970562e+06,6.970562e+06,7.008440e+01,-1.578565e+01,1.436783e+01,1.162371e+01,4.519739e+09,8.137713e+01,6.605466e+01,2.693749e+00,4.421227e+00,6.567086e+01,8.743365e+01,8.217781e+00,1.204507e+01
75%,1.045584e+07,1.045584e+07,1.013998e+02,-1.516850e+01,1.491868e+01,2.869398e+01,6.968809e+09,1.274517e+02,9.401759e+01,3.236852e+01,3.900011e+01,1.380510e+02,1.418556e+02,5.145242e+01,7.072402e+01
max,1.394112e+07,1.394112e+07,6.939670e+06,-1.115399e+01,1.687405e+01,1.713278e+03,9.285641e+09,2.218282e+05,1.801212e+03,2.213788e+05,2.456662e+05,3.379222e+05,3.379440e+05,3.374946e+05,4.890921e+05


In [24]:
agF50.to_csv(os.path.join(out_pth, 'ag_to_HDurban_mmF50.csv'))

In [29]:
# If reloading:
agF10 = os.path.join(out_pth, "ag_to_HDurban_flood10.csv")
agF10 = pd.read_csv(agF10)
print(agF10.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625851 entries, 0 to 625850
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  625851 non-null  int64  
 1   NN          625851 non-null  int64  
 2   ag_HD1F10   625841 non-null  float64
 3   ag_HD2F10   625851 non-null  float64
 4   ag_HD3F10   625851 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 23.9 MB
None


In [30]:
agF10 = agF10[['NN', 'ag_HD1F10']]
agF10

,NN,ag_HD1F10
0,3507831609,82.054823
1,3507831510,88.413319
2,6188134127,23.536822
3,8631201421,94.721696
4,8598305977,51.737958
...,...,...
625846,3651042474,95480.310462
625847,3651042508,95472.160207
625848,3651042501,95477.655323
625849,3651042393,95464.125021


In [31]:
agF50 = os.path.join(out_pth, "ag_to_HDurban_mmF50.csv")
agF50 = pd.read_csv(agF50)
print(agF50.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 17 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   Unnamed: 0.1  int64  
 2   ID_ag         int64  
 3   val           float64
 4   x             float64
 5   y             float64
 6   NSnomax       float64
 7   NN            int64  
 8   F20           float64
 9   base          float64
 10  dif_20b       float64
 11  pc_20b        float64
 12  geometry      object 
 13  ag_HD1F50     float64
 14  F50           float64
 15  dif_50b       float64
 16  pc_50b        float64
dtypes: float64(12), int64(4), object(1)
memory usage: 1.8+ GB
None


In [32]:
agF50 = pd.merge(agF50, agF10, on='NN',how='left')
agF50

,Unnamed: 0,Unnamed: 0.1,ID_ag,val,x,y,NSnomax,NN,F20,base,dif_20b,pc_20b,geometry,ag_HD1F50,F50,dif_50b,pc_50b,ag_HD1F10
0,0,0,13941103,2008851.0,-16.458988,12.208646,NaN,3507831609,NaN,NaN,NaN,NaN,POINT (-16.458988 12.208646),82.731533,NaN,NaN,NaN,82.054823
1,1,1,13941122,94605.0,-16.375655,12.208646,NaN,3507831510,NaN,NaN,NaN,NaN,POINT (-16.375655 12.208646),89.090029,NaN,NaN,NaN,88.413319
2,2,2,13941017,708413.0,-16.208989,12.208646,NaN,6188134127,NaN,NaN,NaN,NaN,POINT (-16.208989 12.208646),23.536822,NaN,NaN,NaN,23.536822
3,3,3,13941036,315719.0,-16.208492,12.152815,NaN,3507831510,NaN,NaN,NaN,NaN,POINT (-16.208492 12.152815),89.090029,NaN,NaN,NaN,88.413319
4,4,4,13941044,375288.0,-16.294213,12.149487,NaN,3507831510,NaN,NaN,NaN,NaN,POINT (-16.294213 12.149487),89.090029,NaN,NaN,NaN,88.413319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13941119,13941119,13941119,13941081,87412.0,-15.291536,16.874047,NaN,3651042474,NaN,NaN,NaN,NaN,POINT (-15.291536 16.874047),186216.753175,NaN,NaN,NaN,95480.310462
13941120,13941120,13941120,13941093,21660.0,-15.292326,16.791961,NaN,3651042508,NaN,NaN,NaN,NaN,POINT (-15.292326 16.791961),186208.602920,NaN,NaN,NaN,95472.160207
13941121,13941121,13941121,13941109,70993.0,-15.374255,16.867753,NaN,3651042501,NaN,NaN,NaN,NaN,POINT (-15.374255 16.867753),186214.098036,NaN,NaN,NaN,95477.655323
13941122,13941122,13941122,13941119,54822.0,-15.375659,16.791961,NaN,3651042393,NaN,NaN,NaN,NaN,POINT (-15.375659 16.791961),186200.567733,NaN,NaN,NaN,95464.125021


In [33]:
agF50['F10'] = 0 # mm for multi-modal
agF50['F10'] = agF50['ag_HD1F10'] + agF50['NSnomax']

In [34]:
# Dif and pc
agF50['dif_10b'] = 0 # dif for difference between the two travel times (in minutes)
agF50['pc_10b'] = 0 # pc for percent change
agF50['dif_10b'] = agF50['F10'] - agF50['base'] # Travel time is X minutes longer in flood conditions.
agF50['pc_10b'] = agF50['dif_10b'] / agF50['base'] * 100 # Travel time is X percent longer in flood conditions.
agF50.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941124 entries, 0 to 13941123
Data columns (total 21 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   Unnamed: 0.1  int64  
 2   ID_ag         int64  
 3   val           float64
 4   x             float64
 5   y             float64
 6   NSnomax       float64
 7   NN            int64  
 8   F20           float64
 9   base          float64
 10  dif_20b       float64
 11  pc_20b        float64
 12  geometry      object 
 13  ag_HD1F50     float64
 14  F50           float64
 15  dif_50b       float64
 16  pc_50b        float64
 17  ag_HD1F10     float64
 18  F10           float64
 19  dif_10b       float64
 20  pc_10b        float64
dtypes: float64(16), int64(4), object(1)
memory usage: 2.3+ GB


In [35]:
# Remove legacy variables.
agF50 = agF50.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'ag_HD1F50', 'geometry', 'ag_HD1F10'])
agF50.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941124 entries, 0 to 13941123
Data columns (total 16 columns):
 #   Column   Dtype  
---  ------   -----  
 0   ID_ag    int64  
 1   val      float64
 2   x        float64
 3   y        float64
 4   NSnomax  float64
 5   NN       int64  
 6   F20      float64
 7   base     float64
 8   dif_20b  float64
 9   pc_20b   float64
 10  F50      float64
 11  dif_50b  float64
 12  pc_50b   float64
 13  F10      float64
 14  dif_10b  float64
 15  pc_10b   float64
dtypes: float64(14), int64(2)
memory usage: 1.8 GB


In [36]:
agF50.to_csv(os.path.join(out_pth, 'ag_to_HDurban_mmF10.csv'))

In [ ]:
print('start: %s\n' % time.ctime())
crs = "EPSG:4326"
geometry = [Point(xy) for xy in zip(agF50.x, agF50.y)]
agF50 = GeoDataFrame(agF50, crs=crs, geometry=geometry) 
agF50.to_file(driver='ESRI Shapefile', filename='C:/Users/wb527163/GEO-Cdrive-Grace/SEN-Cdrive/ag_to_HDurban_base-F10-20-50.shp') 
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Tue Dec 28 14:46:55 2021



#### Hamlets

In [9]:
base_CD = os.path.join(out_pth, "fromhamlet_drive-CD.csv") # Contains base travel time, and the cost-distance travel from origin to road node.
base_CD = pd.read_csv(base_CD)
hamlet_snap = os.path.join(out_pth, "hamlet_snap.csv") 
hamlet_snap = pd.read_csv(hamlet_snap)

print(base_CD.info())
print(hamlet_snap.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125886 entries, 0 to 125885
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   mgrs_code  125886 non-null  object 
 1   GlobalID   125886 non-null  object 
 2   HD1        125886 non-null  float64
 3   NSnomax    125552 non-null  float64
 4   HD1mm      125552 non-null  float64
dtypes: float64(3), object(2)
memory usage: 4.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125886 entries, 0 to 125885
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  125886 non-null  int64  
 1   Unnamed_ 0  125886 non-null  int64  
 2   mgrs_code   125886 non-null  object 
 3   type        125886 non-null  object 
 4   GlobalID    125886 non-null  object 
 5   Shape_Leng  125886 non-null  float64
 6   Shape_Area  125886 non-null  float64
 7   geometry    125886 non-null  object 
 8   NN        

In [19]:
# If reloading:
hamletF10 = os.path.join(out_pth, "hamlet_to_HDurban_flood10.csv")
hamletF10 = pd.read_csv(hamletF10)
hamletF20 = os.path.join(out_pth, "hamlet_to_HDurban_flood20.csv")
hamletF20 = pd.read_csv(hamletF20)
hamletF50 = os.path.join(out_pth, "hamlet_to_HDurban_flood50.csv")
hamletF50 = pd.read_csv(hamletF50)

print(hamletF10.info())
print(hamletF20.info())
print(hamletF50.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61230 entries, 0 to 61229
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  61230 non-null  int64  
 1   NN          61230 non-null  int64  
 2   ha_HD1F10   61227 non-null  float64
 3   ha_HD2F10   61230 non-null  float64
 4   ha_HD3F10   61230 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 2.3 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61230 entries, 0 to 61229
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  61230 non-null  int64  
 1   NN          61230 non-null  int64  
 2   ha_HD1F20   61227 non-null  float64
 3   ha_HD2F20   61230 non-null  float64
 4   ha_HD3F20   61230 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 2.3 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61230 entries, 0 to 61229
Data columns (total 5 co

In [11]:
# Make sure the ID is unique and fully matched
list1 = list(hamletF50['NN'].unique())
list2 = list(hamletF20['NN'].unique())
list3 = list(hamletF10['NN'].unique())
list4 = list(hamlet_snap['NN'].unique())
print(len(list1))
print(len(list2))
print(len(list3))
print(len(list4))

list5 = list(hamlet_snap['mgrs_code'].unique())
list6 = list(base_CD['mgrs_code'].unique())
print(len(list5))
print(len(list6))

61230
61230
61230
61230
125886
125886


In [20]:
hamletF10 = hamletF10[['NN', 'ha_HD1F10']]
hamletF20 = hamletF20[['NN', 'ha_HD1F20']]
hamletF50 = hamletF50[['NN', 'ha_HD1F50']]
hamlet_snap = hamlet_snap[['mgrs_code', 'geometry', 'NN']]
base_CD = base_CD[['mgrs_code', 'NSnomax', 'HD1mm']]

print(hamletF10.head(), end='\n\n')
print(hamletF20.head(), end='\n\n')
print(hamletF50.head(), end='\n\n')
print(hamlet_snap.head(), end='\n\n')
print(base_CD.head(), end='\n\n')

           NN  ha_HD1F10
0  7761872870  62.891073
1  7761872869  62.872513
2  6442044321  61.862681
3  2142496418  63.316203
4  2142496429  63.698577

           NN  ha_HD1F20
0  7761872870  63.403221
1  7761872869  63.384661
2  6442044321  62.374829
3  2142496418  63.828351
4  2142496429  64.210725

           NN  ha_HD1F50
0  7761872870  65.474196
1  7761872869  65.455636
2  6442044321  64.445804
3  2142496418  65.899326
4  2142496429  66.281700

      mgrs_code                                        geometry          NN
0  28PCU1265_01  POINT (-16.721473282454415 12.348636090165247)  7761872870
1  28PCU1365_01   POINT (-16.716456507935607 12.34788723564788)  7761872869
2  28PCU1365_02  POINT (-16.713855008830738 12.350880992129111)  6442044321
3  28PCU1566_01  POINT (-16.701275174874546 12.355585269999269)  2142496418
4  28PCU1566_02  POINT (-16.698773736706396 12.356804484409668)  2142496429

      mgrs_code  NSnomax  HD1mm
0  28PCU1265_01      NaN    NaN
1  28PCU1365_01      NaN  

In [21]:
hamlet_all = pd.merge(hamlet_snap, base_CD, on='mgrs_code',how='left')
print(hamlet_all.info(), end='\n\n')
print(hamlet_all.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   mgrs_code  125886 non-null  object 
 1   geometry   125886 non-null  object 
 2   NN         125886 non-null  int64  
 3   NSnomax    125552 non-null  float64
 4   HD1mm      125552 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 5.8+ MB
None

      mgrs_code                                        geometry          NN  \
0  28PCU1265_01  POINT (-16.721473282454415 12.348636090165247)  7761872870   
1  28PCU1365_01   POINT (-16.716456507935607 12.34788723564788)  7761872869   
2  28PCU1365_02  POINT (-16.713855008830738 12.350880992129111)  6442044321   
3  28PCU1566_01  POINT (-16.701275174874546 12.355585269999269)  2142496418   
4  28PCU1566_02  POINT (-16.698773736706396 12.356804484409668)  2142496429   

   NSnomax  HD1mm  
0      NaN    NaN  
1      NaN    NaN  


In [22]:
hamlet_all = pd.merge(hamlet_all, hamletF10, on='NN',how='left')
print(hamlet_all.info(), end='\n\n')
print(hamlet_all.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   mgrs_code  125886 non-null  object 
 1   geometry   125886 non-null  object 
 2   NN         125886 non-null  int64  
 3   NSnomax    125552 non-null  float64
 4   HD1mm      125552 non-null  float64
 5   ha_HD1F10  125883 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 6.7+ MB
None

      mgrs_code                                        geometry          NN  \
0  28PCU1265_01  POINT (-16.721473282454415 12.348636090165247)  7761872870   
1  28PCU1365_01   POINT (-16.716456507935607 12.34788723564788)  7761872869   
2  28PCU1365_02  POINT (-16.713855008830738 12.350880992129111)  6442044321   
3  28PCU1566_01  POINT (-16.701275174874546 12.355585269999269)  2142496418   
4  28PCU1566_02  POINT (-16.698773736706396 12.356804484409668)  2142496429   

   NSnomax  HD1mm  

In [23]:
hamlet_all = pd.merge(hamlet_all, hamletF20, on='NN',how='left')
print(hamlet_all.info(), end='\n\n')
print(hamlet_all.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   mgrs_code  125886 non-null  object 
 1   geometry   125886 non-null  object 
 2   NN         125886 non-null  int64  
 3   NSnomax    125552 non-null  float64
 4   HD1mm      125552 non-null  float64
 5   ha_HD1F10  125883 non-null  float64
 6   ha_HD1F20  125883 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 7.7+ MB
None

      mgrs_code                                        geometry          NN  \
0  28PCU1265_01  POINT (-16.721473282454415 12.348636090165247)  7761872870   
1  28PCU1365_01   POINT (-16.716456507935607 12.34788723564788)  7761872869   
2  28PCU1365_02  POINT (-16.713855008830738 12.350880992129111)  6442044321   
3  28PCU1566_01  POINT (-16.701275174874546 12.355585269999269)  2142496418   
4  28PCU1566_02  POINT (-16.698773736706396 12.35680448440

In [24]:
hamlet_all = pd.merge(hamlet_all, hamletF50, on='NN',how='left')
print(hamlet_all.info(), end='\n\n')
print(hamlet_all.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   mgrs_code  125886 non-null  object 
 1   geometry   125886 non-null  object 
 2   NN         125886 non-null  int64  
 3   NSnomax    125552 non-null  float64
 4   HD1mm      125552 non-null  float64
 5   ha_HD1F10  125883 non-null  float64
 6   ha_HD1F20  125883 non-null  float64
 7   ha_HD1F50  125883 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 8.6+ MB
None

      mgrs_code                                        geometry          NN  \
0  28PCU1265_01  POINT (-16.721473282454415 12.348636090165247)  7761872870   
1  28PCU1365_01   POINT (-16.716456507935607 12.34788723564788)  7761872869   
2  28PCU1365_02  POINT (-16.713855008830738 12.350880992129111)  6442044321   
3  28PCU1566_01  POINT (-16.701275174874546 12.355585269999269)  2142496418   
4  28PCU1566_02  

In [25]:
hamlet_all.rename(columns={'HD1mm':'base'}, inplace=True)

hamlet_all['F10'] = 0
hamlet_all['F10'] = hamlet_all['ha_HD1F10'] + hamlet_all['NSnomax']
hamlet_all['F20'] = 0
hamlet_all['F20'] = hamlet_all['ha_HD1F20'] + hamlet_all['NSnomax']
hamlet_all['F50'] = 0
hamlet_all['F50'] = hamlet_all['ha_HD1F50'] + hamlet_all['NSnomax']

hamlet_all['dif_10b'] = 0 # dif for difference between the two travel times (in minutes)
hamlet_all['pc_10b'] = 0 # pc for percent change
hamlet_all['dif_10b'] = hamlet_all['F10'] - hamlet_all['base'] # Travel time is X minutes longer in flood conditions.
hamlet_all['pc_10b'] = hamlet_all['dif_10b'] / hamlet_all['base'] * 100 # Travel time is X percent longer in flood conditions.

hamlet_all['dif_20b'] = 0 # dif for difference between the two travel times (in minutes)
hamlet_all['pc_20b'] = 0 # pc for percent change
hamlet_all['dif_20b'] = hamlet_all['F20'] - hamlet_all['base'] # Travel time is X minutes longer in flood conditions.
hamlet_all['pc_20b'] = hamlet_all['dif_20b'] / hamlet_all['base'] * 100 # Travel time is X percent longer in flood conditions.

hamlet_all['dif_50b'] = 0 # dif for difference between the two travel times (in minutes)
hamlet_all['pc_50b'] = 0 # pc for percent change
hamlet_all['dif_50b'] = hamlet_all['F50'] - hamlet_all['base'] # Travel time is X minutes longer in flood conditions.
hamlet_all['pc_50b'] = hamlet_all['dif_50b'] / hamlet_all['base'] * 100 # Travel time is X percent longer in flood conditions.

hamlet_all.head()

,mgrs_code,geometry,NN,NSnomax,base,ha_HD1F10,ha_HD1F20,ha_HD1F50,F10,F20,F50,dif_10b,pc_10b,dif_20b,pc_20b,dif_50b,pc_50b
0,28PCU1265_01,POINT (-16.721473282454415 12.348636090165247),7761872870,NaN,NaN,62.891073,63.403221,65.474196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28PCU1365_01,POINT (-16.716456507935607 12.34788723564788),7761872869,NaN,NaN,62.872513,63.384661,65.455636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28PCU1365_02,POINT (-16.713855008830738 12.350880992129111),6442044321,NaN,NaN,61.862681,62.374829,64.445804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28PCU1566_01,POINT (-16.701275174874546 12.355585269999269),2142496418,NaN,NaN,63.316203,63.828351,65.899326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28PCU1566_02,POINT (-16.698773736706396 12.356804484409668),2142496429,NaN,NaN,63.698577,64.210725,66.281700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
hamlet_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 17 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   mgrs_code  125886 non-null  object 
 1   geometry   125886 non-null  object 
 2   NN         125886 non-null  int64  
 3   NSnomax    125552 non-null  float64
 4   base       125552 non-null  float64
 5   ha_HD1F10  125883 non-null  float64
 6   ha_HD1F20  125883 non-null  float64
 7   ha_HD1F50  125883 non-null  float64
 8   F10        125549 non-null  float64
 9   F20        125549 non-null  float64
 10  F50        125549 non-null  float64
 11  dif_10b    125549 non-null  float64
 12  pc_10b     125548 non-null  float64
 13  dif_20b    125549 non-null  float64
 14  pc_20b     125548 non-null  float64
 15  dif_50b    125549 non-null  float64
 16  pc_50b     125548 non-null  float64
dtypes: float64(14), int64(1), object(2)
memory usage: 17.3+ MB


In [27]:
display(hamlet_all.describe())

,NN,NSnomax,base,ha_HD1F10,ha_HD1F20,ha_HD1F50,F10,F20,F50,dif_10b,pc_10b,dif_20b,pc_20b,dif_50b,pc_50b
count,1.258860e+05,125552.000000,125552.000000,125883.000000,125883.000000,125883.000000,125549.000000,125549.000000,125549.000000,1.255490e+05,1.255480e+05,1.255490e+05,1.255480e+05,1.255490e+05,1.255480e+05
mean,5.308692e+09,23.883785,96.106479,514.512566,1038.752648,2090.903028,521.785551,1035.056472,2069.402565,4.256771e+02,2.509807e+02,9.389480e+02,5.792683e+02,1.973294e+03,1.288904e+03
std,2.359009e+09,31.678803,72.802323,4787.457434,8334.005393,13427.302266,4656.940001,8104.217997,13120.371573,4.644338e+03,2.149234e+03,8.090129e+03,3.797200e+03,1.310162e+04,6.527796e+03
min,2.238784e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.136868e-13,-5.250238e-14,-5.684342e-14,-5.250238e-14,-5.684342e-14,-5.327503e-14
25%,3.670522e+09,5.330675,43.628459,29.890441,31.590684,34.098236,45.576487,48.903011,51.569585,1.812162e-01,3.350923e-01,4.285746e-01,6.610646e-01,8.361956e-01,1.479530e+00
50%,4.635455e+09,11.602750,75.401475,53.324630,63.959088,70.909068,83.663933,94.022616,104.567724,1.693662e+00,2.130879e+00,5.376619e+00,6.068798e+00,1.593465e+01,1.700048e+01
75%,7.798350e+09,30.418225,128.243527,143.970177,178.066406,244.660680,168.715592,203.875029,280.531199,1.481516e+01,1.350110e+01,5.395519e+01,4.627568e+01,1.069482e+02,9.895040e+01
max,9.285336e+09,871.611000,934.181380,142262.045968,215643.754476,329565.199228,142296.201168,215677.909676,329599.354428,1.418084e+05,1.868351e+05,2.151901e+05,1.870044e+05,3.291116e+05,3.038653e+05


In [28]:
# Remove legacy variables.
hamlet_all = hamlet_all.drop(columns=['ha_HD1F10', 'ha_HD1F20', 'ha_HD1F50'])
hamlet_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   mgrs_code  125886 non-null  object 
 1   geometry   125886 non-null  object 
 2   NN         125886 non-null  int64  
 3   NSnomax    125552 non-null  float64
 4   base       125552 non-null  float64
 5   F10        125549 non-null  float64
 6   F20        125549 non-null  float64
 7   F50        125549 non-null  float64
 8   dif_10b    125549 non-null  float64
 9   pc_10b     125548 non-null  float64
 10  dif_20b    125549 non-null  float64
 11  pc_20b     125548 non-null  float64
 12  dif_50b    125549 non-null  float64
 13  pc_50b     125548 non-null  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 14.4+ MB


In [29]:
hamlet_all["geometry"] = hamlet_all["geometry"].astype('str')
hamlet_all["geometry"]  = hamlet_all["geometry"] .str.strip('POINT ')
hamlet_all["geometry"]  = hamlet_all["geometry"] .str.strip('()')
XY = hamlet_all["geometry"] .str.split(" ", expand=True)
hamlet_all["X"] = XY[0]
hamlet_all["Y"] = XY[1]
hamlet_all["X"] = hamlet_all["X"].astype('float')
hamlet_all["Y"] = hamlet_all["Y"].astype('float')
hamlet_all = hamlet_all.drop(columns=['geometry'])
hamlet_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 15 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   mgrs_code  125886 non-null  object 
 1   NN         125886 non-null  int64  
 2   NSnomax    125552 non-null  float64
 3   base       125552 non-null  float64
 4   F10        125549 non-null  float64
 5   F20        125549 non-null  float64
 6   F50        125549 non-null  float64
 7   dif_10b    125549 non-null  float64
 8   pc_10b     125548 non-null  float64
 9   dif_20b    125549 non-null  float64
 10  pc_20b     125548 non-null  float64
 11  dif_50b    125549 non-null  float64
 12  pc_50b     125548 non-null  float64
 13  X          125886 non-null  float64
 14  Y          125886 non-null  float64
dtypes: float64(13), int64(1), object(1)
memory usage: 15.4+ MB


In [30]:
hamlet_all.to_csv(os.path.join(out_pth, 'hamlet_to_HDurban_base-F10-20-50.csv'))

In [31]:
print('start: %s\n' % time.ctime())
crs = "EPSG:4326"
geometry = [Point(xy) for xy in zip(hamlet_all.X, hamlet_all.Y)]
hamlet_all = GeoDataFrame(hamlet_all, crs=crs, geometry=geometry) 
hamlet_all.to_file(driver='ESRI Shapefile', filename='C:/Users/wb527163/GEO-Cdrive-Grace/SEN-Cdrive/hamlet_to_HDurban_base-F10-20-50.shp') 
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Mon Jan  3 14:14:59 2022


end: Mon Jan  3 14:15:56 2022

--- processing complete


#### Agriculture

In [9]:
base_CD = os.path.join(out_pth, "fromag_drive-CD.csv") # Contains base travel time, and the cost-distance travel from origin to road node.
base_CD = pd.read_csv(base_CD)
ag_snap = os.path.join(out_pth, "ag_snap.csv") 
ag_snap = pd.read_csv(ag_snap)

In [10]:
print(base_CD.info())
print(ag_snap.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 5 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ID_ag     int64  
 1   val       float64
 2   NSnomax   float64
 3   NN        int64  
 4   ag_HD1mm  float64
dtypes: float64(3), int64(2)
memory usage: 531.8 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 12 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   ID_ag       int64  
 2   ID_spam     float64
 3   grid_val    float64
 4   ID_LC       float64
 5   val         float64
 6   x           float64
 7   y           float64
 8   NSnomax     float64
 9   geometry    object 
 10  NN          int64  
 11  NN_dist     float64
dtypes: float64(8), int64(3), object(1)
memory usage: 1.2+ GB
None


In [11]:
# If reloading:
agF10 = os.path.join(out_pth, "ag_to_HDurban_flood10.csv")
agF10 = pd.read_csv(agF10)
agF20 = os.path.join(out_pth, "ag_to_HDurban_flood20.csv")
agF20 = pd.read_csv(agF20)
agF50 = os.path.join(out_pth, "ag_to_HDurban_flood50.csv")
agF50 = pd.read_csv(agF50)

print(agF10.info())
print(agF20.info())
print(agF50.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625851 entries, 0 to 625850
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  625851 non-null  int64  
 1   NN          625851 non-null  int64  
 2   ag_HD1F10   625841 non-null  float64
 3   ag_HD2F10   625851 non-null  float64
 4   ag_HD3F10   625851 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 23.9 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 16 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   Unnamed: 0.1  int64  
 2   ID_ag         int64  
 3   ID_spam       float64
 4   grid_val      float64
 5   ID_LC         float64
 6   val           float64
 7   x             float64
 8   y             float64
 9   NSnomax       float64
 10  geometry      object 
 11  NN            int64  
 12  NN_dist       float64
 13  ag_HD1F20     f

In [12]:
# Make sure the ID is unique and fully matched
list1 = list(agF50['NN'].unique())
list2 = list(agF20['NN'].unique())
list3 = list(agF10['NN'].unique())
list4 = list(ag_snap['NN'].unique())
print(len(list1))
print(len(list2))
print(len(list3))
print(len(list4))

list5 = list(ag_snap['ID_ag'].unique())
list6 = list(base_CD['ID_ag'].unique())
print(len(list5))
print(len(list6))

625851
625851
625851
625851
13941124
13941124


In [13]:
agF10 = agF10[['NN', 'ag_HD1F10']]
agF20 = agF20[['NN', 'ag_HD1F20']]
agF50 = agF50[['NN', 'ag_HD1F50']]
ag_snap = ag_snap[['ID_ag', 'geometry', 'NN']]
base_CD = base_CD[['ID_ag', 'val', 'NSnomax', 'ag_HD1mm']]

print(agF10.head(), end='\n\n')
print(agF20.head(), end='\n\n')
print(agF50.head(), end='\n\n')
print(ag_snap.head(), end='\n\n')
print(base_CD.head(), end='\n\n')

           NN  ag_HD1F10
0  3507831609  82.054823
1  3507831510  88.413319
2  6188134127  23.536822
3  8631201421  94.721696
4  8598305977  51.737958

           NN  ag_HD1F20
0  3507831609  82.054823
1  3507831510  88.413319
2  6188134127  23.536822
3  3507831510  88.413319
4  3507831510  88.413319

           NN   ag_HD1F50
0  3507831609   82.731533
1  3507831510   89.090029
2  6188134127   23.536822
3  8631201421  503.912228
4  8598305977  518.180056

      ID_ag                                        geometry          NN
0  13941103  POINT (-16.458987999999977 12.208646000000044)  3507831609
1  13941122  POINT (-16.375654999999938 12.208646000000044)  3507831510
2  13941017  POINT (-16.208988999999974 12.208646000000044)  6188134127
3  13941036   POINT (-16.20849199999998 12.152815000000032)  3507831510
4  13941044  POINT (-16.294212999999957 12.149487000000022)  3507831510

      ID_ag        val  NSnomax  ag_HD1mm
0  13941103  2008851.0      NaN       NaN
1  13941122    94605.0  

In [14]:
ag_all = pd.merge(ag_snap, base_CD, on='ID_ag',how='left')
print(ag_all.info(), end='\n\n')
print(ag_all.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941124 entries, 0 to 13941123
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ID_ag     int64  
 1   geometry  object 
 2   NN        int64  
 3   val       float64
 4   NSnomax   float64
 5   ag_HD1mm  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 744.5+ MB
None

      ID_ag                                        geometry          NN  \
0  13941103  POINT (-16.458987999999977 12.208646000000044)  3507831609   
1  13941122  POINT (-16.375654999999938 12.208646000000044)  3507831510   
2  13941017  POINT (-16.208988999999974 12.208646000000044)  6188134127   
3  13941036   POINT (-16.20849199999998 12.152815000000032)  3507831510   
4  13941044  POINT (-16.294212999999957 12.149487000000022)  3507831510   

         val  NSnomax  ag_HD1mm  
0  2008851.0      NaN       NaN  
1    94605.0      NaN       NaN  
2   708413.0      NaN       NaN  
3   315719.0      NaN       NaN  
4   375288.

In [15]:
agF10 = pd.merge(ag_all, agF10, on='NN',how='left')
print(ag_all.info(), end='\n\n')
print(ag_all.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941124 entries, 0 to 13941123
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   ID_ag      int64  
 1   geometry   object 
 2   NN         int64  
 3   val        float64
 4   NSnomax    float64
 5   ag_HD1mm   float64
 6   ag_HD1F10  float64
dtypes: float64(4), int64(2), object(1)
memory usage: 850.9+ MB
None

      ID_ag                                        geometry          NN  \
0  13941103  POINT (-16.458987999999977 12.208646000000044)  3507831609   
1  13941122  POINT (-16.375654999999938 12.208646000000044)  3507831510   
2  13941017  POINT (-16.208988999999974 12.208646000000044)  6188134127   
3  13941036   POINT (-16.20849199999998 12.152815000000032)  3507831510   
4  13941044  POINT (-16.294212999999957 12.149487000000022)  3507831510   

         val  NSnomax  ag_HD1mm  ag_HD1F10  
0  2008851.0      NaN       NaN  82.054823  
1    94605.0      NaN       NaN  88.413319  
2   708413.0  

In [17]:
# Save intermediate file, because it's so huge.
agF10.to_csv(os.path.join(out_pth, 'ag_to_HDurban_base-F10.csv'))

In [9]:
# Restart kernel from here if having memory errors.
agF10 = os.path.join(out_pth, "ag_to_HDurban_base-F10.csv")
agF10 = pd.read_csv(agF10)

In [13]:
# Due to memory errors, need to create different data files for each flood.
agF10.rename(columns={'ag_HD1mm':'base'}, inplace=True)
agF10['F10'] = 0
agF10['F10'] = agF10['ag_HD1F10'] + agF10['NSnomax']
agF10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   ID_ag       int64  
 2   geometry    object 
 3   NN          int64  
 4   val         float64
 5   NSnomax     float64
 6   base        float64
 7   ag_HD1F10   float64
 8   F10         float64
dtypes: float64(5), int64(3), object(1)
memory usage: 957.3+ MB


In [14]:
display(agF10.describe())

,Unnamed: 0,ID_ag,NN,val,NSnomax,base,ag_HD1F10,F10
count,1.394112e+07,1.394112e+07,1.394112e+07,1.394112e+07,1.318626e+07,1.318596e+07,1.394082e+07,1.318596e+07
mean,6.970562e+06,6.970562e+06,5.121305e+09,1.219702e+02,2.209557e+01,7.234398e+01,8.373060e+02,2.889649e+02
std,4.024456e+06,4.024456e+06,2.262019e+09,3.760062e+03,2.796064e+01,4.196269e+01,7.494902e+03,2.940325e+03
min,0.000000e+00,0.000000e+00,2.214129e+07,1.563126e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.485281e+06,3.485281e+06,3.656862e+09,4.293236e+01,5.456861e+00,4.336144e+01,2.983436e+01,4.542230e+01
50%,6.970562e+06,6.970562e+06,4.519739e+09,7.008440e+01,1.162371e+01,6.605466e+01,4.840991e+01,7.124638e+01
75%,1.045584e+07,1.045584e+07,6.968809e+09,1.013998e+02,2.869398e+01,9.401759e+01,9.097461e+01,1.084964e+02
max,1.394112e+07,1.394112e+07,9.285641e+09,6.939670e+06,1.713278e+03,1.801212e+03,1.527663e+05,1.527882e+05


In [15]:
# Remove legacy variables.
agF10 = agF10.drop(columns=['Unnamed: 0','ag_HD1F10'])
agF10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 7 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ID_ag     int64  
 1   geometry  object 
 2   NN        int64  
 3   val       float64
 4   NSnomax   float64
 5   base      float64
 6   F10       float64
dtypes: float64(4), int64(2), object(1)
memory usage: 744.5+ MB


In [16]:
# Save to file
agF10.to_csv(os.path.join(out_pth, 'ag_to_HDurban_base-F10.csv'))

1 in 20 year flood.

In [14]:
# Restart kernel from here if having memory errors.
agF10 = os.path.join(out_pth, "ag_to_HDurban_base-F10.csv")
agF10 = pd.read_csv(agF10)
agF10 = agF10.drop(columns=['Unnamed: 0', 'geometry', 'val', 'F10'])

In [15]:
agF10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   ID_ag    int64  
 1   NN       int64  
 2   NSnomax  float64
 3   base     float64
dtypes: float64(2), int64(2)
memory usage: 425.4 MB


In [16]:
agF20 = os.path.join(out_pth, "ag_to_HDurban_flood20.csv")
agF20 = pd.read_csv(agF20)
agF20 = agF20[['NN', 'ag_HD1F20']]

agF20 = pd.merge(agF10, agF20, on='NN',how='left')
print(agF20.info(), end='\n\n')
print(agF20.head())

MemoryError: Unable to allocate 107. GiB for an array with shape (14399955670,) and data type int64

In [17]:
agF20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 2 columns):
 #   Column     Dtype  
---  ------     -----  
 0   NN         int64  
 1   ag_HD1F20  float64
dtypes: float64(1), int64(1)
memory usage: 212.7 MB


In [ ]:
agF20.to_csv(os.path.join(out_pth, 'ag_to_HDurban_base-F20.csv'))

In [ ]:
# Due to memory errors, need to create different data files for each flood.
agF20['F20'] = 0
agF20['F20'] = agF20['ag_HD1F20'] + agF20['NSnomax']
agF20.info()

In [ ]:
# Restart kernel from here if having memory errors.
ag_all = os.path.join(out_pth, "ag_to_HDurban_base-F10-20-50.csv")
ag_all = pd.read_csv(ag_all)
agF50 = os.path.join(out_pth, "ag_to_HDurban_flood50.csv")
agF50 = pd.read_csv(agF50)
agF50 = agF20[['NN', 'ag_HD1F50']]

In [ ]:
ag_all = pd.merge(ag_all, agF50, on='NN',how='left')
print(ag_all.info(), end='\n\n')
print(ag_all.head())
ag_all.to_csv(os.path.join(out_pth, 'ag_to_HDurban_base-F10-20-50.csv'))

In [ ]:
ag_all['F20'] = 0
ag_all['F20'] = ag_all['ha_HD1F20'] + ag_all['NSnomax']
ag_all['F50'] = 0
ag_all['F50'] = ag_all['ha_HD1F50'] + ag_all['NSnomax']

In [ ]:
# Remove legacy variables.
ag_all = ag_all.drop(columns=['ha_HD1F10', 'ha_HD1F20', 'ha_HD1F50'])
ag_all.info()

In [ ]:
ag_all["geometry"] = ag_all["geometry"].astype('str')
ag_all["geometry"]  = ag_all["geometry"] .str.strip('POINT ')
ag_all["geometry"]  = ag_all["geometry"] .str.strip('()')
XY = ag_all["geometry"] .str.split(" ", expand=True)
ag_all["X"] = XY[0]
ag_all["Y"] = XY[1]
ag_all["X"] = ag_all["X"].astype('float')
ag_all["Y"] = ag_all["Y"].astype('float')
ag_all = ag_all.drop(columns=['geometry'])
ag_all.info()

In [ ]:
ag_all.to_csv(os.path.join(out_pth, 'ag_to_HDurban_base-F10-20-50.csv'))

In [ ]:
print('start: %s\n' % time.ctime())
crs = "EPSG:4326"
geometry = [Point(xy) for xy in zip(ag_all.X, ag_all.Y)]
ag_all = GeoDataFrame(ag_all, crs=crs, geometry=geometry) 
ag_all.to_file(driver='ESRI Shapefile', filename='C:/Users/wb527163/GEO-Cdrive-Grace/SEN-Cdrive/ag_to_HDurban_base-F10-20-50.shp') 
print('\nend: %s' % time.ctime())
print('\n--- processing complete')